In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

In [2]:
def detect_outliers_iqr(data, column):
    q1 = data[column].quantile(0.25)
    q3 = data[column].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    return data[(data[column] < lower_bound) | (data[column] > upper_bound)]

### Working on Customers

In [3]:

# Create SQLAlchemy engine
engine = create_engine("mysql+pymysql://root:@localhost:3306/diet_center_analysis")

df_customers = pd.read_sql_query("SELECT * FROM customers", engine)

In [4]:
df_customers

,id,first_name,middle_name,last_name,username,phone,email,email_verified_at,password,profile_image,...,prospect_id,height,weight,verify_code,is_verify,enable_notify,remember_token,created_at,updated_at,deleted_at
0,4,Rami,Salim,Alhalaby,rami,99898543,None,None,$2y$10$8JAl2IqAdYwZwYdHeAGZTumEIX970J851RxgoF5...,None,...,SAMA-045752,184,80,965185,1,0,None,2022-05-26 17:42:03,2025-05-14 11:09:12,NaT
1,5,Omar,Imad,Naccache,omar,99507181,None,None,$2y$10$L41NL/ezd/tKtJ6ZI0jHT.vv/sVPCVbOvbWhKZJ...,None,...,SAMA-045839,0,0,687132,1,1,None,2022-06-01 10:06:38,2023-08-19 21:07:41,NaT
2,6,Waleed,Khaled,Al Shammari,waleedkhaled,99081281,None,None,$2y$10$3mjVipTjmaqdbUYbPiEL/uvu6xsL52PYp9G0ShR...,None,...,SAMA-037412,0,0,661543,1,1,None,2022-06-07 10:58:15,2023-09-18 16:49:27,NaT
3,7,Tareq,Jabr,Mohamed,tareq,66646675,None,None,$2y$10$ez6w9u8GziM41oGUzWMHAOEZkp5sqCRsijho5ie...,None,...,SAMA-035878,0,0,1,1,1,None,2022-06-07 11:09:37,2022-06-07 11:09:55,NaT
4,8,Fahed,Naif,Almutari,FAHAD,50866088,None,None,$2y$10$B.GgeZPgUuFKds6Y3rSoUu2fksCMGE.uTAe4fRW...,None,...,SAMA-019226,0,0,175875,1,1,None,2022-06-07 11:11:17,2022-08-28 10:07:04,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8027,8203,ahmad,ali,alshanfa,ahmadshanfa,55514589,ahmadshanfa91@gmail.com,None,$2y$10$szRNzhVRWjerHoxPANkZA.KqDqn08NtwdVtN8aC...,None,...,SAMA-061199,175,110,569480,1,1,None,2025-05-20 10:22:39,2025-05-20 10:23:30,NaT
8028,8204,Deem,Hamed,Alazmi,dodo,55194014,itsdeema444@gmail.com,None,$2y$10$xq/YAAci8ut2/YgvUSlOd.LVWnTz3ahl2iwYy1Z...,None,...,SAMA-061200,155,60,533659,1,1,None,2025-05-20 11:13:12,2025-05-20 11:13:33,NaT
8029,8205,Abeer,Fahad,Alazmi,Abeer Fahad,94055774,None,None,$2y$10$b8S2cAHG1w8N.neP0XvNbO8Ms9VMvPfd/Ic8zn5...,None,...,SAMA-061201,161,82,149748,1,1,None,2025-05-20 12:56:35,2025-05-20 12:57:21,NaT
8030,8206,Mnahy,Khaled,Aldehani,mnahy,60707024,Thebicvv644@gmail.com,None,$2y$10$mQY2Kgzoz4CI4j5qYoWQFOEO4sWLqqMoKxJkiPi...,None,...,SAMA-061202,170,97,None,0,1,None,2025-05-20 13:08:03,2025-05-20 13:08:03,NaT


In [5]:
df_customers.columns

Index(['id', 'first_name', 'middle_name', 'last_name', 'username', 'phone',
       'email', 'email_verified_at', 'password', 'profile_image', 'referral',
       'civil_id', 'nationality_id', 'gender', 'date_of_birth', 'age',
       'customer_id', 'prospect_id', 'height', 'weight', 'verify_code',
       'is_verify', 'enable_notify', 'remember_token', 'created_at',
       'updated_at', 'deleted_at'],
      dtype='object')

In [6]:
pd.read_sql_query("SELECT * FROM nationalities", engine)

,id,name,name_en,code,created_at,updated_at
0,1,الإمارات العربية المتحدة,United Arab Emirates,ARE,None,None
1,2,الأرجنتين,Argentina,ARG,None,None
2,3,أرمينيا,Armenia,ARM,None,None
3,4,أستراليا,Australia,AUS,None,None
4,5,البحرين,Bahrain,BHR,None,None
5,6,البرازيل,Brazil,BRA,None,None
6,7,كندا,Canada,CAN,None,None
7,8,الصين,China,CHN,None,None
8,9,ألمانيا,Germany,DEU,None,None
9,10,الدنمارك,Denmark,DNK,None,None


In [7]:
#Select important columns with nationalities
df_customers = pd.read_sql_query(""" SELECT c.id, c.username, c.email, 
                                        n.name_en nationality, c.gender, c.date_of_birth, TIMESTAMPDIFF(YEAR, c.date_of_birth, CURDATE()) AS age,
                                        c.height, c.weight, c.created_at, c.deleted_at 
                                    FROM customers c INNER JOIN nationalities n ON c.nationality_id = n.id
                                    """, engine)

In [8]:
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8028 entries, 0 to 8027
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             8028 non-null   int64         
 1   username       8028 non-null   object        
 2   email          5270 non-null   object        
 3   nationality    8028 non-null   object        
 4   gender         8028 non-null   object        
 5   date_of_birth  8028 non-null   object        
 6   age            8028 non-null   int64         
 7   height         8028 non-null   object        
 8   weight         8028 non-null   object        
 9   created_at     8027 non-null   datetime64[ns]
 10  deleted_at     14 non-null     datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 690.0+ KB


In [9]:
df_customers.duplicated().sum()

0

In [10]:
# number of deleted customers
df_customers['deleted_at'].notnull().sum()

14

In [11]:
# customers who deleted their accounts
df_customers[df_customers['deleted_at'].notnull()]

,id,username,email,nationality,gender,date_of_birth,age,height,weight,created_at,deleted_at
151,7443,S O,None,Algeria,female,2000-01-01,25,158,60,2024-12-17 13:15:45,2025-03-12 00:32:47
581,6819,MoElmorsy,Mo.elsheikh92@gmail.com,Egypt,male,2024-07-30,0,171,102,2024-07-29 20:18:06,2024-07-30 08:54:41
4337,4487,Adel123,gbtix@hotmail.com,Kuwait,male,2024-07-05,1,170,77,2023-07-12 21:53:24,2024-07-20 12:33:55
4839,5127,Mohsen1,mm99455466@gmail.com,Kuwait,male,1985-03-16,40,171,100,2023-09-23 02:20:43,2024-10-01 19:22:10
5848,6498,albatoul baqer,tootabaqer.99@gmail.com,Kuwait,female,1999-12-12,25,157,79,2024-06-08 23:08:27,2024-12-07 21:40:03
6104,6809,hamda,alhryjyha88@gamil.com,Kuwait,female,1988-07-09,37,152,84,2024-07-28 01:04:47,2024-07-28 01:08:43
6593,7435,Analysis Mohammed,None,Kuwait,female,2001-06-27,24,0,0,2024-12-15 18:40:06,2024-12-15 20:32:52
6619,7473,Eman Ali,65199542,Kuwait,female,1989-04-09,36,168,60,2024-12-24 09:40:32,2025-05-03 15:47:36
6800,7705,hsx29,hayakfm96@gmail.com,Kuwait,female,2000-01-01,25,160,77,2025-01-28 19:41:45,2025-01-28 20:46:41
6831,7745,eman alobaied,manialkhashti@gmail.com,Kuwait,female,1964-02-06,61,152,92,2025-02-04 12:16:36,2025-02-04 14:17:37


In [12]:
# convert date_of_birth to datetime
df_customers['date_of_birth'] = pd.to_datetime(df_customers['date_of_birth'], format='%Y-%m-%d')
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8028 entries, 0 to 8027
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             8028 non-null   int64         
 1   username       8028 non-null   object        
 2   email          5270 non-null   object        
 3   nationality    8028 non-null   object        
 4   gender         8028 non-null   object        
 5   date_of_birth  8028 non-null   datetime64[ns]
 6   age            8028 non-null   int64         
 7   height         8028 non-null   object        
 8   weight         8028 non-null   object        
 9   created_at     8027 non-null   datetime64[ns]
 10  deleted_at     14 non-null     datetime64[ns]
dtypes: datetime64[ns](3), int64(2), object(6)
memory usage: 690.0+ KB


In [13]:
# convert height and weight to numeric
df_customers['height'] = pd.to_numeric(df_customers['height'], errors='coerce')
df_customers['weight'] = pd.to_numeric(df_customers['weight'], errors='coerce')
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8028 entries, 0 to 8027
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             8028 non-null   int64         
 1   username       8028 non-null   object        
 2   email          5270 non-null   object        
 3   nationality    8028 non-null   object        
 4   gender         8028 non-null   object        
 5   date_of_birth  8028 non-null   datetime64[ns]
 6   age            8028 non-null   int64         
 7   height         8028 non-null   int64         
 8   weight         8028 non-null   int64         
 9   created_at     8027 non-null   datetime64[ns]
 10  deleted_at     14 non-null     datetime64[ns]
dtypes: datetime64[ns](3), int64(4), object(4)
memory usage: 690.0+ KB


In [14]:
df_customers['birth_year'] = df_customers['date_of_birth'].dt.year
df_customers['birth_month'] = df_customers['date_of_birth'].dt.month
df_customers['birth_day'] = df_customers['date_of_birth'].dt.day
df_customers['birth_weekday'] = df_customers['date_of_birth'].dt.day_name()
df_customers['birth_quarter'] = df_customers['date_of_birth'].dt.quarter
df_customers['created_year'] = df_customers['created_at'].dt.year
df_customers['created_month'] = df_customers['created_at'].dt.month
df_customers['created_month_name'] = df_customers['created_at'].dt.month_name()
df_customers['created_day'] = df_customers['created_at'].dt.day
df_customers['created_weekday'] = df_customers['created_at'].dt.day_name()
df_customers['created_quarter'] = df_customers['created_at'].dt.quarter
df_customers

,id,username,email,nationality,gender,date_of_birth,age,height,weight,created_at,...,birth_month,birth_day,birth_weekday,birth_quarter,created_year,created_month,created_month_name,created_day,created_weekday,created_quarter
0,687,tarekmagdi,tarekmagdi81@gmail.com,United Arab Emirates,male,1988-07-24,36,175,95,2022-07-06 12:50:03,...,7,24,Sunday,3,2022.0,7.0,July,6.0,Wednesday,3.0
1,3768,sultan1,None,United Arab Emirates,male,1994-05-18,31,174,93,2023-05-02 13:26:41,...,5,18,Wednesday,2,2023.0,5.0,May,2.0,Tuesday,2.0
2,5179,ayeshafardan,ayeshafardan@gmail.com,United Arab Emirates,female,1989-03-15,36,173,127,2023-09-28 10:21:07,...,3,15,Wednesday,1,2023.0,9.0,September,28.0,Thursday,3.0
3,5824,Hananalshibli,Hananals@outlook.com,United Arab Emirates,female,1986-02-15,39,146,56,2024-02-03 19:26:15,...,2,15,Saturday,1,2024.0,2.0,February,3.0,Saturday,1.0
4,2563,aqeel,aqeel.boushehri@gmail.com,Argentina,male,1989-03-21,36,177,110,2022-11-22 21:57:19,...,3,21,Tuesday,1,2022.0,11.0,November,22.0,Tuesday,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8023,6848,khalidmajid,majid78@hotmail.com,Pakistan,male,1978-01-09,47,189,135,2024-08-05 10:22:59,...,1,9,Monday,1,2024.0,8.0,August,5.0,Monday,3.0
8024,6868,syedusama,usamanajeeb003@gmail.com,Pakistan,male,1993-08-28,31,194,59,2024-08-09 13:10:09,...,8,28,Saturday,3,2024.0,8.0,August,9.0,Friday,3.0
8025,7421,Anita,None,Pakistan,female,1966-02-25,59,0,0,2024-12-10 18:45:16,...,2,25,Friday,1,2024.0,12.0,December,10.0,Tuesday,4.0
8026,7676,arwa98,None,Pakistan,female,1998-06-09,27,163,62,2025-01-24 21:43:16,...,6,9,Tuesday,2,2025.0,1.0,January,24.0,Friday,1.0


In [15]:
#customers by nationality
nationality_order = df_customers['nationality'].value_counts().index.tolist()
px.histogram(
    df_customers, 
    x='nationality', 
    title='Number of Customers by Nationality',
    text_auto=True,
    category_orders={'nationality': nationality_order}
)

In [16]:
# replace all nationality with counts <9 to others
nationality_percent = df_customers['nationality'].value_counts(normalize=True) * 100
nationality_percent

nationality
Kuwait                  77.167414
Egypt                    5.231689
Lebanon                  4.197808
Jordan                   2.665670
Syria                    1.681614
Algeria                  1.507225
Saudi Arabia             1.145989
Non Kuwaiti              1.108620
India                    0.934230
Others                   0.585451
Palestine                0.485800
Canada                   0.423518
Iran                     0.398605
United States            0.336323
Armenia                  0.286497
Iraq                     0.224215
Philippines              0.186846
United Kingdom           0.186846
Pakistan                 0.161933
Yemen                    0.124564
Venezuela                0.112108
Morocco                  0.074738
Turkey                   0.074738
Tunisia                  0.074738
Australia                0.062282
France                   0.062282
Bahrain                  0.062282
Brazil                   0.062282
United Arab Emirates     0.049826
Su

In [17]:
nationality_counts = df_customers['nationality'].value_counts()
to_replace = nationality_counts[nationality_counts <= 9].index
df_customers['nationality'] = df_customers['nationality'].replace(to_replace, 'Others')
df_customers['nationality'].value_counts(normalize=True) * 100

nationality
Kuwait            77.167414
Egypt              5.231689
Lebanon            4.197808
Jordan             2.665670
Syria              1.681614
Others             1.544594
Algeria            1.507225
Saudi Arabia       1.145989
Non Kuwaiti        1.108620
India              0.934230
Palestine          0.485800
Canada             0.423518
Iran               0.398605
United States      0.336323
Armenia            0.286497
Iraq               0.224215
United Kingdom     0.186846
Philippines        0.186846
Pakistan           0.161933
Yemen              0.124564
Name: proportion, dtype: float64

In [18]:
# customers by age
px.histogram(
    df_customers, 
    x='age', 
    title='Number of Customers by Age',
    nbins=30,
    text_auto=True
).update_layout(bargap=0.2)

In [19]:
df_customers['age'].describe()

count    8028.000000
mean       30.916044
std        12.883081
min         0.000000
25%        25.000000
50%        31.000000
75%        39.000000
max       125.000000
Name: age, dtype: float64

In [20]:
px.violin(
    df_customers, 
    x='age', 
    box=True, 
    points='all', 
    title='Age Distribution of Customers'
).update_layout(yaxis_title='Age')

In [21]:
# check if this customer is correct and not deleted also
abnormal_ages = df_customers[ df_customers['age'] > 80][['id', 'deleted_at']]
abnormal_ages['id']

1141     223
4553    4766
7470    6789
7960    1977
Name: id, dtype: int64

In [22]:
# check if the abnormal ages have customers programs
abnormal_customer_ids = tuple(abnormal_ages['id'])

if len(abnormal_customer_ids) == 1:
    abnormal_customer_ids = (abnormal_customer_ids[0], abnormal_customer_ids[0])

query = f"""
SELECT * FROM customers_programs
WHERE customer_id IN {abnormal_customer_ids}
"""
pd.read_sql_query(query, engine)
# this customer has programs

,id,customer_id,program_id,promocode_id,paid,paid_amount,payment_method,payment_reference,delivery_start_date,total_days,free_days,status,created_at,updated_at
0,5637276517,223,5637167080,5637144582,1,135.0,DCKNETMOBILE,2022002755,19/06/2022,26,0,4,2022-06-14 08:24:27,2022-06-19 10:01:06
1,5637281649,1977,5637167077,0,1,155.0,None,None,03/09/2022,28,2,2,2022-09-29 11:47:16,2023-01-04 21:16:26
2,5637282846,1977,5637167077,0,1,155.0,None,None,01/10/2022,26,0,2,2022-09-29 11:47:16,2023-01-04 21:16:27
3,5637319269,6789,5637177579,5637198576,1,115.0,None,None,21/07/2024,22,0,2,2024-07-25 13:55:17,2024-08-19 13:24:10


In [23]:
# customers registered in each year
px.histogram(
    df_customers, 
    x='created_year', 
    title='Number of Customers Registered in Each Year',
    text_auto=True,
).update_layout(bargap=0.2)

In [24]:
# percentage of customers registered in each year
px.pie(
    df_customers, 
    names='created_year', 
    title='Percentage of Customers Registered in Each Year')

In [25]:
# customers registered in each month
px.histogram(
    df_customers, 
    x='created_month_name', 
    title='Number of Customers Registered in Each Month',
    category_orders={'created_month_name': ['January', 'February', 'March', 'April', 'May', 'June', 
                                            'July', 'August', 'September', 'October', 'November', 'December']},
    text_auto=True,
    color='gender'
).update_layout(bargap=0.2)

In [26]:
# most popular days for customer registration
px.histogram(
    df_customers, 
    x='created_weekday', 
    title='Most Popular Days for Customer Registration',
    category_orders={'created_weekday': ['Saturday', 'Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']},
    text_auto=True,
    color='gender'
).update_layout(bargap=0.2)

In [27]:
# age with nationality
avg_age = df_customers.groupby('nationality')['age'].mean().reset_index()
px.bar(
    avg_age,
    y='nationality',
    x='age',
    title='Average Age by Nationality',
    text_auto=True
)

In [28]:
df_customers.describe()

,id,date_of_birth,age,height,weight,created_at,deleted_at,birth_year,birth_month,birth_day,birth_quarter,created_year,created_month,created_day,created_quarter
count,8028.000000,8028,8028.000000,8028.000000,8028.000000,8027,14,8028.000000,8028.000000,8028.000000,8028.000000,8027.000000,8027.000000,8027.000000,8027.000000
mean,4124.828849,1994-02-15 13:49:25.022421504,30.916044,125.488416,65.053189,2023-07-27 05:02:03.568580864,2025-01-08 03:52:20.785714432,1993.630917,6.488540,15.152591,2.504484,2023.029027,6.946680,16.698019,2.658777
min,4.000000,1900-01-01 00:00:00,0.000000,0.000000,0.000000,2022-05-26 17:42:03,2024-07-20 12:33:55,1900.000000,1.000000,1.000000,1.000000,2022.000000,1.000000,1.000000,1.000000
25%,2113.750000,1986-06-27 00:00:00,25.000000,77.000000,45.000000,2022-10-09 17:16:16,2024-10-18 13:56:38.249999872,1986.000000,4.000000,8.000000,2.000000,2022.000000,5.000000,9.000000,2.000000
50%,4128.500000,1994-03-12 12:00:00,31.000000,161.000000,74.000000,2023-06-16 08:03:17,2025-02-01 05:32:09,1994.000000,7.000000,15.000000,3.000000,2023.000000,7.000000,18.000000,3.000000
75%,6136.250000,2000-05-08 12:00:00,39.000000,171.000000,90.000000,2024-04-26 03:08:43.500000,2025-04-10 07:28:34,2000.000000,9.000000,23.000000,3.000000,2024.000000,9.000000,25.000000,3.000000
max,8207.000000,2025-05-19 00:00:00,125.000000,888.000000,888.000000,2025-05-20 14:20:26,2025-05-19 07:58:17,2025.000000,12.000000,31.000000,4.000000,2025.000000,12.000000,31.000000,4.000000
std,2360.357366,NaN,12.883081,73.486971,45.759997,NaN,NaN,12.871791,3.436334,8.854907,1.103289,0.949395,3.064088,8.898688,0.988999


In [29]:
df_customers.select_dtypes(include=['number']).corr()

,id,age,height,weight,birth_year,birth_month,birth_day,birth_quarter,created_year,created_month,created_day,created_quarter
id,1.000000,-0.055922,0.063983,0.049918,0.057208,-0.045159,-0.032598,-0.040023,0.938937,-0.125472,0.023699,-0.092140
age,-0.055922,1.000000,-0.117312,-0.081286,-0.999253,-0.021327,-0.045698,-0.027766,-0.059600,0.016077,-0.018146,0.011540
height,0.063983,-0.117312,1.000000,0.810872,0.117106,0.008460,0.002037,0.012400,0.044354,0.013130,0.016226,0.014134
weight,0.049918,-0.081286,0.810872,1.000000,0.081117,0.003319,-0.007924,0.009255,0.036654,0.000617,0.013051,0.004190
birth_year,0.057208,-0.999253,0.117106,0.081117,1.000000,-0.011549,0.042092,-0.004906,0.062136,-0.020302,0.018658,-0.015749
birth_month,-0.045159,-0.021327,0.008460,0.003319,-0.011549,1.000000,0.038872,0.970883,-0.080776,0.125608,-0.011937,0.122441
birth_day,-0.032598,-0.045698,0.002037,-0.007924,0.042092,0.038872,1.000000,0.041138,-0.037421,0.001570,0.040393,-0.004718
birth_quarter,-0.040023,-0.027766,0.012400,0.009255,-0.004906,0.970883,0.041138,1.000000,-0.075250,0.123622,-0.011367,0.126559
created_year,0.938937,-0.059600,0.044354,0.036654,0.062136,-0.080776,-0.037421,-0.075250,1.000000,-0.411069,-0.013489,-0.372278
created_month,-0.125472,0.016077,0.013130,0.000617,-0.020302,0.125608,0.001570,0.123622,-0.411069,1.000000,0.015608,0.962672


In [30]:
df_customers.describe(include='O')

,username,email,nationality,gender,birth_weekday,created_month_name,created_weekday
count,8028,5270,8028,8028,8028,8027,8027
unique,8028,5270,20,3,7,12,7
top,tarekmagdi,tarekmagdi81@gmail.com,Kuwait,female,Saturday,June,Sunday
freq,1,1,6195,4287,1290,1114,1244


In [31]:
# extract domain from email
df_customers['email_domain'] = df_customers['email'].str.split('@').str[1]
df_customers['email_domain'].value_counts()

email_domain
gmail.com             3153
hotmail.com           1166
icloud.com             216
yahoo.com              207
outlook.com            147
                      ... 
yahoo.com.au             1
hotmail.de               1
kic-kw.com               1
waw.cc                   1
mailam-shaalan.com       1
Name: count, Length: 141, dtype: int64

In [32]:
# display the top 10 email domains
df_customers['email_domain'].value_counts().head(20)

email_domain
gmail.com          3153
hotmail.com        1166
icloud.com          216
yahoo.com           207
outlook.com         147
live.com             74
gmail.con            24
gamil.com            17
outlook.sa           14
windowslive.com      12
gmail                11
me.com                9
hotmail.con           7
hotmail.co.uk         6
yahoo.co.uk           5
gmil.com              5
msn.com               4
mail.com              3
gmial.com             3
ymail.com             3
Name: count, dtype: int64

In [33]:
# visualize the customers by gender
px.histogram(
    df_customers,
    x='gender',
    title='Number of Customers per Gender',
    text_auto=True
).update_layout(bargap=0.2)

In [34]:
px.pie(df_customers,
       names='gender',
       title='Percentage of Customers by age')

In [35]:
# visualize the customers using gender with nationalities
px.histogram(
    df_customers,
    x='nationality',
    color='gender',
    title='Number of Customers by Nationality and Gender',
    text_auto=True,
    category_orders={'nationality': df_customers['nationality'].value_counts().index.tolist()}
).update_layout(bargap=0.2)

In [36]:
df_customers['weight'].isnull().sum()

0

In [37]:
df_customers['height'].isnull().sum()

0

In [38]:
# box plot for height
px.box(
    df_customers,
    x='height',
    title='Height Distribution of Customers',
    points='all'
).update_layout(yaxis_title='Height (cm)')

In [39]:
px.histogram(
    df_customers,
    x='weight',
    title='Weight Distribution of Customers',
    marginal='box'
).update_layout(yaxis_title='Weight (kg)')

In [40]:
# using detect_outliers_iqr function to detect outliers in height and weight
df_customers_height_outliers = detect_outliers_iqr(df_customers, 'height').sort_values('height')
df_customers_weight_outliers = detect_outliers_iqr(df_customers, 'weight').sort_values('weight')
display(df_customers_weight_outliers['weight'].value_counts(), df_customers_height_outliers['height'].value_counts())


weight
160    11
165     7
175     5
170     5
168     4
158     4
190     3
162     3
180     3
173     3
178     2
166     2
186     2
159     2
195     2
200     2
161     2
167     2
500     1
456     1
809     1
303     1
830     1
254     1
250     1
210     1
205     1
873     1
185     1
199     1
193     1
189     1
188     1
187     1
181     1
177     1
176     1
172     1
169     1
163     1
888     1
Name: count, dtype: int64

height
512    1
541    1
598    1
769    1
888    1
Name: count, dtype: int64

In [41]:
# replace outliers in height and weight with the meadian
df_customers['height'] = df_customers['height'].mask(df_customers['height'].isin(df_customers_height_outliers['height']), df_customers['height'].median())
df_customers['weight'] = df_customers['weight'].mask(df_customers['weight'].isin(df_customers_weight_outliers['weight']), df_customers['weight'].median())

In [42]:
df_customers_height_outliers = detect_outliers_iqr(df_customers, 'height').sort_values('height')
df_customers_weight_outliers = detect_outliers_iqr(df_customers, 'weight').sort_values('weight')
display(df_customers_weight_outliers['weight'].value_counts(), df_customers_height_outliers['height'].value_counts())

Series([], Name: count, dtype: int64)

Series([], Name: count, dtype: int64)

In [43]:
# visualize the height & weight using kdeplot
fig = ff.create_distplot(
    [df_customers['height'].dropna(), df_customers['weight'].dropna()],
    ['Height', 'Weight'],
    show_hist=False,
    show_rug=False
)
fig.update_layout(    title='Height and Weight Distribution of Customers',
    xaxis_title='Value',
    yaxis_title='Density'
)
fig.show()

In [44]:
# visualize the height & weight using distplot
display(px.histogram(
    df_customers,
    x='height',
    title='Height Distribution of Customers',
    marginal='box',
    nbins=30
).update_layout(yaxis_title='Height (cm)'),
px.histogram(
    df_customers,
    x='weight',
    title='Weight Distribution of Customers',
    marginal='box',
    nbins=30
).update_layout(yaxis_title='Weight (kg)')
)

In [45]:
# dealing with zeros values in height and weight by depending on age & gender
df_customers['height'] = df_customers['height'].replace(0, np.nan)
df_customers['weight'] = df_customers['weight'].replace(0, np.nan)

In [46]:
le = LabelEncoder()
df_customers['gender_encoded'] = le.fit_transform(df_customers['gender'])

In [47]:
imputer = KNNImputer(n_neighbors=5)
df_imputed = pd.DataFrame(imputer.fit_transform(df_customers[['age', 'height', 'weight', 'gender_encoded']]), columns=['age', 'height', 'weight', 'gender_encoded'])
df_customers['age'] = df_imputed['age']
df_customers['height'] = df_imputed['height']
df_customers['weight'] = df_imputed['weight']
display(px.histogram(
    df_customers,
    x='height',
    title='Height Distribution of Customers',
    marginal='box',
    nbins=30
).update_layout(yaxis_title='Height (cm)'),
px.histogram(
    df_customers,
    x='weight',
    title='Weight Distribution of Customers',
    marginal='box',
    nbins=30
).update_layout(yaxis_title='Weight (kg)')
)

In [48]:
df_customers['height'].describe()

count    8028.000000
mean      164.656677
std        16.479562
min         1.000000
25%       159.000000
50%       165.000000
75%       174.000000
max       202.000000
Name: height, dtype: float64

In [49]:
# replace outliers in height and weight with the meadian
df_customers_height_outliers = detect_outliers_iqr(df_customers, 'height').sort_values('height')
df_customers_weight_outliers = detect_outliers_iqr(df_customers, 'weight').sort_values('weight')
df_customers['height'] = df_customers['height'].mask(df_customers['height'].isin(df_customers_height_outliers['height']), df_customers['height'].median())
df_customers['weight'] = df_customers['weight'].mask(df_customers['weight'].isin(df_customers_weight_outliers['weight']), df_customers['weight'].median())

In [50]:
display(px.histogram(
    df_customers,
    x='height',
    title='Height Distribution of Customers',
    marginal='box',
    nbins=30
).update_layout(yaxis_title='Height (cm)'),
px.histogram(
    df_customers,
    x='weight',
    title='Weight Distribution of Customers',
    marginal='box',
    nbins=30
).update_layout(yaxis_title='Weight (kg)')
)

In [51]:
# calculate BMI
df_customers['bmi'] = df_customers['weight'] / ((df_customers['height'] / 100) ** 2)

In [52]:
df_customers['bmi'].isnull().sum()

0

In [53]:
df_customers[df_customers['bmi'].isnull()]['height'].unique()

array([], dtype=float64)

In [54]:
df_customers[df_customers['bmi'].isnull()]['weight'].unique()

array([], dtype=float64)

In [55]:
px.box(
    df_customers,
    x='bmi',
    title='BMI Distribution of Customers',
    points='all'
).update_layout(yaxis_title='BMI')

In [56]:
df_customers[['age', 'height', 'weight', 'bmi']].corr()

,age,height,weight,bmi
age,1.000000,0.053991,0.103992,0.084592
height,0.053991,1.000000,0.476810,-0.118181
weight,0.103992,0.476810,1.000000,0.809832
bmi,0.084592,-0.118181,0.809832,1.000000


In [57]:
px.scatter_matrix(
    df_customers,
    dimensions=['age', 'height', 'weight', 'bmi'],
    title='Scatter Matrix of Age, Height, Weight, and BMI')

# show registration customers per month over all years with line plot

In [58]:
df_customers['created_month_year_str'] = df_customers['created_at'].dt.strftime('%b-%Y')

monthly_counts = df_customers.groupby('created_month_year_str').size().reset_index(name='count')

monthly_counts['created_month_year_str'] = pd.to_datetime(monthly_counts['created_month_year_str'], format='%b-%Y')
monthly_counts = monthly_counts.sort_values('created_month_year_str')
monthly_counts['created_month_year_str'] = monthly_counts['created_month_year_str'].dt.strftime('%b-%Y')

px.line(
    monthly_counts,
    x='created_month_year_str',
    y='count',
    title='Number of Customers Registered Each Month',
    markers=True,
    text='count',
).update_layout(
    xaxis_title='Month',
    yaxis_title='Number of Customers',
    xaxis_tickangle=-45
)

In [59]:
df_customers['height'].value_counts()

height
165.0    467
160.0    413
170.0    413
158.0    264
175.0    249
        ... 
154.2      1
139.4      1
178.2      1
149.2      1
171.6      1
Name: count, Length: 121, dtype: int64

In [60]:
df_customers.to_csv('../2.clean-data/customers_cleaned.csv', index=False)

# Customers Addresses

In [61]:
df_customer_addresses = pd.read_sql_query("SELECT * FROM customers_address WHERE deleted_at IS NULL", engine)
df_customer_addresses

,id,ax_name,ax_id,name,customer_id,governorate_id,city_id,block,street,avenue,building,flat,floor,additional_comments,is_default,type,created_at,updated_at,deleted_at
0,8,ADD 0,5637237897,Home,5,5637144578,5637144616,10,9,None,None,None,None,Building: 285 | Flat: 2,1,3,2022-06-01 10:06:39,2022-06-01 10:20:54,None
1,10,Add1,5637221653,,7,5637144580,5637144580,2,5,None,None,None,None,Building 2,0,3,2022-06-07 11:09:38,2022-06-07 11:09:38,None
2,11,Add2,5637221654,,7,5637144577,5637144626,4,5,None,None,None,None,House 14,0,3,2022-06-07 11:09:38,2022-06-07 11:09:38,None
3,12,ADD2,5637241354,,8,5637144577,5637144626,3,4,None,None,None,None,house: 50 - floor 3 - flat :4 / Evening Delive...,0,3,2022-06-07 11:11:18,2022-06-07 11:11:18,None
4,13,add1,5637239834,,9,5637144580,5637144585,1,144,None,None,None,None,home 11 / floor 2 / Apt 1 / the beside door,1,3,2022-06-07 11:23:20,2022-06-07 11:23:20,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5468,5824,ADD1,5637274463,,8191,5637144579,5637144677,6,619,None,None,None,None,H 24,0,3,2025-05-19 21:25:36,2025-05-19 21:25:36,None
5469,5825,ADD 0,5637274465,Home,8194,5637144580,5637144589,2,20,None,1 B,1 B,None,House: 1 B / Blue House / call the client on ...,1,3,2025-05-19 21:46:28,2025-05-20 00:56:39,None
5470,5826,ADD 1,5637274466,Alaa,8158,5637144578,5637144607,7,287,None,21,36,None,Building: 21 / House: 36 / Dont ring the bel...,0,3,2025-05-19 23:45:13,2025-05-20 00:56:37,None
5471,5827,Add1,5637274442,,8158,5637144578,5637144607,7,street 281 behind al bahar center,None,None,None,None,building 21 opposite fresh mart floor 3 flat 3...,0,3,2025-05-20 13:56:40,2025-05-20 13:56:40,None


In [62]:
# get addresses with governorates, cities, customers and nationalities
df_customer_addresses = pd.read_sql_query("""SELECT * FROM customers_address ca  
                                          INNER JOIN governorates g on ca.governorate_id = g.id
                                          INNER JOIN cities c on c.id = ca.city_id
                                          INNER JOIN customers c2 on c2.id = ca.customer_id
                                          WHERE ca.deleted_at IS NULL and c2.deleted_at IS NULL""", engine)
df_customer_addresses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5461 entries, 0 to 5460
Data columns (total 60 columns):
 #   Column               Non-Null Count  Dtype          
---  ------               --------------  -----          
 0   id                   5461 non-null   int64          
 1   ax_name              5450 non-null   object         
 2   ax_id                5450 non-null   object         
 3   name                 5461 non-null   object         
 4   customer_id          5461 non-null   int64          
 5   governorate_id       5461 non-null   int64          
 6   city_id              5461 non-null   int64          
 7   block                5461 non-null   object         
 8   street               5461 non-null   object         
 9   avenue               577 non-null    object         
 10  building             1799 non-null   object         
 11  flat                 1799 non-null   object         
 12  floor                0 non-null      object         
 13  additional_comment

In [63]:
# drop unnecessary columns
df_customer_addresses = pd.read_sql_query("""SELECT ca.id, ca.ax_id, ca.block, ca.street, ca.building, 
                                          ca.flat, ca.floor, ca.is_default, ca.type, g.name_en AS governorate_name,
                                          c.name_en AS city_name, c.from_time AS city_from_time, c.to_time AS city_to_time,
                                          n.name_en AS nationality_name
                                          FROM customers_address ca 
                                          INNER JOIN governorates g on ca.governorate_id = g.id
                                          INNER JOIN cities c on c.id = ca.city_id
                                          INNER JOIN customers c2 on c2.id = ca.customer_id
                                          INNER JOIN nationalities n on c2.nationality_id = n.id
                                          WHERE ca.deleted_at IS NULL AND c2.deleted_at IS NULL""", engine)
df_customer_addresses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5451 entries, 0 to 5450
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype          
---  ------            --------------  -----          
 0   id                5451 non-null   int64          
 1   ax_id             5440 non-null   object         
 2   block             5451 non-null   object         
 3   street            5451 non-null   object         
 4   building          1797 non-null   object         
 5   flat              1797 non-null   object         
 6   floor             0 non-null      object         
 7   is_default        5451 non-null   int64          
 8   type              5451 non-null   object         
 9   governorate_name  5451 non-null   object         
 10  city_name         5451 non-null   object         
 11  city_from_time    5450 non-null   timedelta64[ns]
 12  city_to_time      5450 non-null   timedelta64[ns]
 13  nationality_name  5451 non-null   object         
dtypes: int64

In [64]:
# map type 1 = flat, 2 = office, 3 = house
df_customer_addresses['type'] = df_customer_addresses['type'].astype(int, errors='ignore')
df_customer_addresses['type'] = df_customer_addresses['type'].map({1: 'Flat', 2: 'Office', 3: 'House'})

In [65]:
df_customer_addresses['type'].value_counts()

type
House     5066
Flat       341
Office      44
Name: count, dtype: int64

In [66]:
df_customer_addresses['city_from_time'] = df_customer_addresses['city_from_time'].astype(str).str.replace('0 days ', '', regex=False)
df_customer_addresses['city_to_time'] = df_customer_addresses['city_to_time'].astype(str).str.replace('0 days ', '', regex=False)

In [67]:
df_cities = df_customer_addresses[['city_name', 'city_from_time', 'city_to_time']]
df_cities = df_cities.drop_duplicates().reset_index(drop=True)
df_cities

,city_name,city_from_time,city_to_time
0,Salwa,08:00:00,12:00:00
1,Dasma,08:00:00,12:00:00
2,Ardhiya,16:00:00,20:00:00
3,Jaber Al Ahmed,12:00:00,17:00:00
4,Reggai,07:00:00,08:00:00
...,...,...,...
101,Ardiya Small Industrial,08:00:00,12:00:00
102,Sabhan Industrial,08:00:00,12:00:00
103,Mubarakiya Camps,09:00:00,12:00:00
104,Al-Bedae,12:00:00,16:00:00


In [68]:
# visualize the cities depending on the from and to time
px.histogram(
    df_cities, 
    x='city_from_time', 
    title='Cities by From Time',
    text_auto=True,
).update_layout(bargap=0.2)

In [69]:
# visualize the cities depending on the from and to time
px.histogram(
    df_cities, 
    x='city_to_time', 
    title='Cities by To Time',
    text_auto=True,
).update_layout(bargap=0.2)

In [70]:
df_cities['city_from_to_time'] = df_cities['city_from_time'] + ' - ' + df_cities['city_to_time']
df_cities['city_from_to_time']

0      08:00:00 - 12:00:00
1      08:00:00 - 12:00:00
2      16:00:00 - 20:00:00
3      12:00:00 - 17:00:00
4      07:00:00 - 08:00:00
              ...         
101    08:00:00 - 12:00:00
102    08:00:00 - 12:00:00
103    09:00:00 - 12:00:00
104    12:00:00 - 16:00:00
105    08:00:00 - 12:00:00
Name: city_from_to_time, Length: 106, dtype: object

In [71]:
px.histogram(
    df_cities, 
    x='city_from_to_time', 
    title='Cities by From and To Time',
    text_auto=True,
).update_layout(bargap=0.2)

In [72]:
df_customer_addresses['city_from_to_time'] = df_customer_addresses['city_from_time'] + ' - ' + df_customer_addresses['city_to_time']
df_customer_addresses

,id,ax_id,block,street,building,flat,floor,is_default,type,governorate_name,city_name,city_from_time,city_to_time,nationality_name,city_from_to_time
0,8,5637237897,10,9,None,None,None,1,House,Hawally,Salwa,08:00:00,12:00:00,Lebanon,08:00:00 - 12:00:00
1,10,5637221653,2,5,None,None,None,0,House,Kuwait,Dasma,08:00:00,12:00:00,Syria,08:00:00 - 12:00:00
2,11,5637221654,4,5,None,None,None,0,House,Farwaniya,Ardhiya,16:00:00,20:00:00,Syria,16:00:00 - 20:00:00
3,12,5637241354,3,4,None,None,None,0,House,Farwaniya,Ardhiya,16:00:00,20:00:00,Kuwait,16:00:00 - 20:00:00
4,13,5637239834,1,144,None,None,None,1,House,Kuwait,Jaber Al Ahmed,12:00:00,17:00:00,Iran,12:00:00 - 17:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5446,5824,5637274463,6,619,None,None,None,0,House,Jahra,Saad Al Abdullah,12:00:00,17:00:00,Kuwait,12:00:00 - 17:00:00
5447,5825,5637274465,2,20,1 B,1 B,None,1,House,Kuwait,Mansouriya,08:00:00,12:00:00,Kuwait,08:00:00 - 12:00:00
5448,5826,5637274466,7,287,21,36,None,0,House,Hawally,Hawally,08:00:00,12:00:00,Kuwait,08:00:00 - 12:00:00
5449,5827,5637274442,7,street 281 behind al bahar center,None,None,None,0,House,Hawally,Hawally,08:00:00,12:00:00,Kuwait,08:00:00 - 12:00:00


In [73]:
# visualize the address per address type
px.histogram(
    df_customer_addresses, 
    x='type', 
    title='Number of Addresses by Type',
    text_auto=True,
).update_layout(bargap=0.2)

In [74]:
# visualize the address depending on the city_from_to_time
px.histogram(
    df_customer_addresses, 
    x='city_from_to_time', 
    title='Addresses by City From and To Time',
    text_auto=True,
).update_layout(bargap=0.2)

In [75]:
df_customer_addresses[ df_customer_addresses['city_from_to_time'] == 'NaT - NaT' ]['city_from_to_time'] = df_customer_addresses['city_from_to_time'].mode()

In [76]:
# visualize nationalities depending on the governoate
px.histogram(
    df_customer_addresses, 
    x='governorate_name',
    color='nationality_name',
    title='Number of Addresses by Governorate',
    text_auto=True,
).update_layout(bargap=0.2)

In [77]:
# check the values of street, building, flat, floor
df_customer_addresses[['street', 'building', 'flat', 'floor']]

,street,building,flat,floor
0,9,None,None,None
1,5,None,None,None
2,5,None,None,None
3,4,None,None,None
4,144,None,None,None
...,...,...,...,...
5446,619,None,None,None
5447,20,1 B,1 B,None
5448,287,21,36,None
5449,street 281 behind al bahar center,None,None,None


In [78]:
df_customer_addresses[['street', 'building', 'flat', 'floor']].isnull().sum()

street         0
building    3654
flat        3654
floor       5451
dtype: int64

In [79]:
# drop the building, flat, floor columns
df_customer_addresses.drop(columns=['building', 'flat', 'floor'], axis=1, inplace=True)
df_customer_addresses

,id,ax_id,block,street,is_default,type,governorate_name,city_name,city_from_time,city_to_time,nationality_name,city_from_to_time
0,8,5637237897,10,9,1,House,Hawally,Salwa,08:00:00,12:00:00,Lebanon,08:00:00 - 12:00:00
1,10,5637221653,2,5,0,House,Kuwait,Dasma,08:00:00,12:00:00,Syria,08:00:00 - 12:00:00
2,11,5637221654,4,5,0,House,Farwaniya,Ardhiya,16:00:00,20:00:00,Syria,16:00:00 - 20:00:00
3,12,5637241354,3,4,0,House,Farwaniya,Ardhiya,16:00:00,20:00:00,Kuwait,16:00:00 - 20:00:00
4,13,5637239834,1,144,1,House,Kuwait,Jaber Al Ahmed,12:00:00,17:00:00,Iran,12:00:00 - 17:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
5446,5824,5637274463,6,619,0,House,Jahra,Saad Al Abdullah,12:00:00,17:00:00,Kuwait,12:00:00 - 17:00:00
5447,5825,5637274465,2,20,1,House,Kuwait,Mansouriya,08:00:00,12:00:00,Kuwait,08:00:00 - 12:00:00
5448,5826,5637274466,7,287,0,House,Hawally,Hawally,08:00:00,12:00:00,Kuwait,08:00:00 - 12:00:00
5449,5827,5637274442,7,street 281 behind al bahar center,0,House,Hawally,Hawally,08:00:00,12:00:00,Kuwait,08:00:00 - 12:00:00


In [80]:
df_customer_addresses.groupby(['governorate_name', 'city_name', 'street']).size().reset_index(name='count')


,governorate_name,city_name,street,count
0,Ahmadi,Abu Halifa,,1
1,Ahmadi,Abu Halifa,17,2
2,Ahmadi,Abu Halifa,20,2
3,Ahmadi,Abu Halifa,209,2
4,Ahmadi,Abu Halifa,24,1
...,...,...,...,...
2855,Mubarak K,Sabah Al-Salem,Mohammed Bin Tahoos St,1
2856,Mubarak K,Sabah Al-Salem,The First / 1,1
2857,Mubarak K,Sabah Al-Salem,الأول,1
2858,Mubarak K,Sabah Al-Salem,٢,1


# Working on Diet Programs

In [81]:
# working on master plans
df_master_plans = pd.read_sql_query("SELECT * FROM masterplans", engine)
df_master_plans

,id,name,name_en,description,description_en,image,display_order,created_at,updated_at,is_active
0,7200,الكامل,Full,۳ وجبات رئيسية + ۲ سلطة أو شوربة + ۲ سناك,3 Mains + 2 Salad/Soup + 2 Snacks,28325cd4cef9eeb948d71fe52876fafc.JPG,100,2023-05-04 14:30:29,2023-12-02 10:24:30,0
1,7300,برنامج الصيف 5,Sayf 5 Plan,3 وجبات رئيسية + 1 سلطة أو شوربة + 1 سناك,3 Mains + 1 Salad or Soup + 1 Snack,afc524a0a10992d537cc687ca50e669b.png,100,2023-02-08 13:01:55,2023-12-02 10:24:30,0
2,7400,برنامج الصيف 7,Sayf 7 Plan,3 وجبات رئيسية + 2 سلطة أو شوربة + 2 سناك,3 Mains + 2 Salads or Soups + 2 Snacks,eb9d66a0a8bfabcf2c9d035a12e186cd.png,100,2023-04-13 13:39:30,2023-12-02 10:24:30,0
3,7500,برنامج الصيف 7 بلس,Sayf 7 Plus Plan,3 وجبات رئيسية + 2 سلطة أو شوربة + 2 سناك (سعر...,3 Mains + 2 Salads or Soups + 2 Snacks (Cal. 1...,9e701d13de50ea74f8eb4ae13fb81217.png,100,2023-04-13 13:41:02,2023-12-02 10:24:30,0
4,7700,رمضان - الكامل,Ramadan - Full,مشروب رمضاني + إفطار (وجبة رئيسية وشوربة) + غب...,Drink + Iftar (Main & Soup) + Ghabqa + Suhour ...,57011973f06dacf1a74dbe013b59df8f.jpg,10012,2023-03-08 14:25:12,2023-05-25 15:44:31,0
...,...,...,...,...,...,...,...,...,...,...
84,13940,لايف 5,Life 5,ريوق + غدا + عشا + 2 سناك,Breakfast + Lunch + Dinner + 2 Snacks,4439793945821ee3355fbfc2c50b319f.jpg,3,2024-12-11 15:18:13,2024-12-15 08:04:18,1
85,13941,لايف 6,Life 6,ريوق + غدا + عشا + 3 سناك,Breakfast + Lunch + Dinner + 3 Snacks,46e99e05f4db75b8776a00aba77307ad.jpg,4,2024-12-11 15:18:47,2024-12-20 10:48:16,1
86,13942,لايف 7,Life 7,ريوق + غدا + عشا + 4 سناك,Breakfast + Lunch + Dinner + 4 Snacks,27b4254a8367a0120ba4ef35c9e35f20.jpg,5,2024-12-11 15:19:18,2024-12-20 10:48:16,1
87,13943,لايف 4,Life 4,ريوق أو سناك + غدا + عشا + سناك,Breakfast or Snack + Lunch + Dinner + 4 Snacks,7523231d80eda3559978feb45c2aa5b4.jpg,2,2024-12-19 15:12:57,2024-12-20 10:48:16,1


In [82]:
# df_master_plans.drop(['description', 'description_en', 'image', 'updated_at'], axis=1, inplace=True)
# df_master_plans

In [83]:
# check for duplicates in master plans depending on name_en
df_master_plans['name_en'].nunique()

88

In [84]:
df_master_plans['name_en'].value_counts()

name_en
Weight Loss                       2
Full                              1
LIFE 5 MEDICAL                    1
HALA SEHHA                        1
LIFE 7 MEDICAL                    1
                                 ..
Wanasa                            1
World Cup                         1
LIFE                              1
Protein 150g (3 Mains + Snack)    1
Life 3                            1
Name: count, Length: 88, dtype: int64

In [85]:
df_master_plans[df_master_plans['name_en'].duplicated(keep=False)]

,id,name,name_en,description,description_en,image,display_order,created_at,updated_at,is_active
7,8000,نزول الوزن,Weight Loss,3 وجبات رئيسية + سلطة أو شوربة + سناك,3 Mains + Salad or Soup + Snack,0297e8df5286ebcd664787653dc1326c.jpeg,100,2023-01-25 08:17:59,2023-12-02 10:24:30,0
28,10100,نزول الوزن,Weight Loss,3 وجبات رئيسية + سلطة أو شوربة + سناك,3 Mains + Salad or Soup + Snack,6f276ded021e62ae05a68c5b0ab807b4.jpg,10016,2022-09-15 12:29:16,2023-05-25 15:50:26,0


In [86]:
# check for duplicates in master plans depending on name
df_master_plans['name'].nunique()

84

In [87]:
df_master_plans['name'].value_counts()

name
لايف 3                          2
لايف 5                          2
هلا صحة                         2
نزول الوزن                      2
لايف 7                          2
                               ..
وناسة                           1
كأس العالم                      1
لايف                            1
بروتين 150غ (3 وجبات + سناك)    1
لايف 4                          1
Name: count, Length: 84, dtype: int64

In [88]:
# select all master plans which have the same name, where value counts is greater than 1
df_master_plans[df_master_plans['name'].duplicated(keep=False)]

,id,name,name_en,description,description_en,image,display_order,created_at,updated_at,is_active
6,7900,هلا صحة,Hala Sehha,3 وجبات رئيسية + سلطة وشوربة + 2 سناك,3 Mains + Salad & Soup + 2 Snacks,4a0246b0d4b95990bc964ed7ef9036bd.png,10013,2023-01-31 10:27:50,2023-05-25 15:44:42,0
7,8000,نزول الوزن,Weight Loss,3 وجبات رئيسية + سلطة أو شوربة + سناك,3 Mains + Salad or Soup + Snack,0297e8df5286ebcd664787653dc1326c.jpeg,100,2023-01-25 08:17:59,2023-12-02 10:24:30,0
28,10100,نزول الوزن,Weight Loss,3 وجبات رئيسية + سلطة أو شوربة + سناك,3 Mains + Salad or Soup + Snack,6f276ded021e62ae05a68c5b0ab807b4.jpg,10016,2022-09-15 12:29:16,2023-05-25 15:50:26,0
53,13909,لايف 3,LIFE 3,وفر 30 دك | كود الخصم: SAVE30\r\nافطار أو طبق ...,Save 30 KD | Promocode: SAVE40\r\nBreakfast or...,ea16fae288450c777b3d2e0ebc7c00e6.jpg,3,2023-11-29 15:51:16,2024-04-03 12:57:45,0
54,13910,لايف 5,LIFE 5,وفر 40 دك | كود الخصم: SAVE40\r\nافطار + سناك ...,Save 40 KD | Promocode: SAVE40\r\nBreakfast + ...,542f44b79abf557a0b244346eecd4681.jpg,2,2023-11-29 15:55:14,2024-04-01 09:02:56,0
58,13914,لايف 7,LIFE 7,وفر 50دك | كود الخصم: SAVE50\r\nافطار + 2 سناك...,Save 50 KD | Promocode: SAVE50\r\nBreakfast + ...,b1f175345e2f01a72e26e0a672d92c37.jpg,1,2023-11-29 16:11:35,2024-04-01 14:13:56,0
65,13921,هلا صحة,HALA SEHHA,افطار + سناك + غداء + سلطة\شوربة + عشاء,Breakfast + Snack + Lunch + Salad/Soup + Dinner,040b9764fadaba320f0a2f4d84947c20.jpg,1,2024-01-21 15:50:13,2024-04-06 18:10:28,0
84,13940,لايف 5,Life 5,ريوق + غدا + عشا + 2 سناك,Breakfast + Lunch + Dinner + 2 Snacks,4439793945821ee3355fbfc2c50b319f.jpg,3,2024-12-11 15:18:13,2024-12-15 08:04:18,1
86,13942,لايف 7,Life 7,ريوق + غدا + عشا + 4 سناك,Breakfast + Lunch + Dinner + 4 Snacks,27b4254a8367a0120ba4ef35c9e35f20.jpg,5,2024-12-11 15:19:18,2024-12-20 10:48:16,1
88,13944,لايف 3,Life 3,ريوق أو سناك + غدا + عشا,Breakfast or Snack + Lunch + Dinner,54406fd6f0fc8cf85a181cb66f46f01c.jpg,1,2024-12-20 10:43:30,2024-12-20 10:48:16,1


### check for duplicates of master plans at level of diet programs


In [89]:
df_diet_programs = pd.read_sql_query("SELECT * FROM diet_programs", engine)
df_diet_programs

,id,master_plan_id,diet_program_calender_id,related_item_id,name,name_en,valid_from,valid_to,calories_total,program_days,day_price,total_amount,created_at,updated_at,is_renewable,display_order
0,5637163328,8700.0,5637164078,None,None,Healthy Keto 30 Days,2022-01-01,2024-12-31,1500,30,9.167,275.001,2022-05-26 11:02:00,2023-05-04 17:18:30,1,0
1,5637163329,8700.0,5637164079,None,None,Healthy Keto 15 Days,2022-01-01,2024-12-31,1500,15,9.933,149.000,2022-05-26 11:02:00,2023-05-04 17:18:30,1,0
2,5637167076,11500.0,5637167076,None,None,Life Full Plan 22 Days | 2022,2022-05-02,2023-12-31,1000,22,8.409,185.000,2022-05-11 00:31:31,2023-01-25 18:33:43,1,0
3,5637167077,11500.0,5637167077,None,None,Life Full Plan 26 Days | 2022,2022-05-02,2023-12-31,1000,26,7.885,205.000,2022-05-11 00:31:31,2023-01-25 18:33:43,1,0
4,5637167078,11500.0,5637167078,None,None,Life Full Plan 30 Days | 2022,2022-05-02,2023-12-31,1000,30,7.833,234.999,2022-05-11 00:31:31,2023-01-25 18:33:43,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,5637180598,13944.0,5637181350,None,None,Life 3 | 26 Days 2025,2024-12-22,2026-12-31,950,26,3.423,102.000,2024-12-19 15:15:29,2025-01-22 14:29:28,1,0
232,5637180599,13944.0,5637181351,None,None,Life 3 | 20 Days 2025,2024-12-22,2026-12-31,950,20,3.950,91.000,2024-12-19 12:29:33,2025-01-22 14:29:19,1,0
233,5637180600,13943.0,5637181352,None,None,Life 4 | 20 Days 2025,2024-12-22,2026-12-31,1000,20,4.700,108.000,2024-12-19 12:23:39,2025-01-22 14:30:56,1,0
234,5637180601,13943.0,5637181353,None,None,Life 4 | 26 Days 2025,2024-12-22,2026-12-31,1000,26,4.192,125.000,2024-12-19 12:26:38,2025-01-22 14:30:48,0,0


### the relation between master plans and diet programs is one to many so can't find the same program in two different master plans

In [90]:
# drop unnecessary columns
df_master_plans.drop(['description', 'image', 'updated_at'], axis=1, inplace=True)

In [91]:
df_master_plans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89 entries, 0 to 88
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              89 non-null     int64         
 1   name            89 non-null     object        
 2   name_en         89 non-null     object        
 3   description_en  88 non-null     object        
 4   display_order   89 non-null     int64         
 5   created_at      89 non-null     datetime64[ns]
 6   is_active       89 non-null     int64         
dtypes: datetime64[ns](1), int64(3), object(3)
memory usage: 5.0+ KB


In [92]:
### visualize the master plans depending on is_active
px.histogram(
    df_master_plans,
    x='is_active',
    title='Number of Master Plans by Active Status',
    text_auto=True
).update_layout(bargap=0.2)

In [93]:
### visualize the master plans depending on display_order and is_active
px.histogram(
    df_master_plans,
    x='display_order',
    color='is_active',
    title='Master Plans by Display Order and Active Status',
    text_auto=True
).update_layout(bargap=0.2)

In [94]:
# visaulize the master plans depending on display_order and is_active, but only for active plans
# to see if the display order reflected on number of subscriptions on these plans
px.histogram(
    df_master_plans[ df_master_plans['is_active'] == 1],
    x='display_order',
    title='Master Plans by Display Order and Active Status',
    text_auto=True,
    nbins=20
).update_layout(bargap=0.2)

In [95]:
# show the master plans with total of diet programs
df_diet_programs.groupby('master_plan_id').size().reset_index(name='total_programs')
df_master_plans = df_master_plans.merge(
    df_diet_programs.groupby('master_plan_id').size().reset_index(name='total_programs'),
    left_on='id',
    right_on='master_plan_id',
    how='inner'
).drop('master_plan_id', axis=1)

In [96]:
df_master_plans

,id,name,name_en,description_en,display_order,created_at,is_active,total_programs
0,7200,الكامل,Full,3 Mains + 2 Salad/Soup + 2 Snacks,100,2023-05-04 14:30:29,0,2
1,7300,برنامج الصيف 5,Sayf 5 Plan,3 Mains + 1 Salad or Soup + 1 Snack,100,2023-02-08 13:01:55,0,3
2,7400,برنامج الصيف 7,Sayf 7 Plan,3 Mains + 2 Salads or Soups + 2 Snacks,100,2023-04-13 13:39:30,0,3
3,7500,برنامج الصيف 7 بلس,Sayf 7 Plus Plan,3 Mains + 2 Salads or Soups + 2 Snacks (Cal. 1...,100,2023-04-13 13:41:02,0,3
4,7700,رمضان - الكامل,Ramadan - Full,Drink + Iftar (Main & Soup) + Ghabqa + Suhour ...,10012,2023-03-08 14:25:12,0,3
...,...,...,...,...,...,...,...,...
83,13940,لايف 5,Life 5,Breakfast + Lunch + Dinner + 2 Snacks,3,2024-12-11 15:18:13,1,2
84,13941,لايف 6,Life 6,Breakfast + Lunch + Dinner + 3 Snacks,4,2024-12-11 15:18:47,1,2
85,13942,لايف 7,Life 7,Breakfast + Lunch + Dinner + 4 Snacks,5,2024-12-11 15:19:18,1,2
86,13943,لايف 4,Life 4,Breakfast or Snack + Lunch + Dinner + 4 Snacks,2,2024-12-19 15:12:57,1,2


In [97]:
# visualize the master plans depending on total_programs
px.histogram(
    df_master_plans,
    x='total_programs',
    title='Number of Master Plans by Total Diet Programs',
    text_auto=True
).update_layout(bargap=0.2)

## Diet Programs Analysis

In [98]:
# diet programs by program days
df_diet_programs['program_days'].value_counts().to_frame(name='count').reset_index().rename(columns={'index': 'program_days'})

,program_days,count
0,26,90
1,22,75
2,30,43
3,20,13
4,7,4
5,12,4
6,5,3
7,15,1
8,6,1
9,14,1


In [99]:
# display all diet programs with program days <= 7
df_diet_programs[df_diet_programs['program_days'] <= 7].sort_values('program_days')

,id,master_plan_id,diet_program_calender_id,related_item_id,name,name_en,valid_from,valid_to,calories_total,program_days,day_price,total_amount,created_at,updated_at,is_renewable,display_order
235,5637181326,NaN,5637182076,None,None,The Test Box 2025,2024-12-22,2026-12-31,1500,1,7.000,7.0,2024-12-19 12:20:38,2025-04-10 12:12:05,1,0
106,5637171577,13907.0,5637171577,None,None,Life - Five 5 Days | 2023,2023-01-25,2024-12-31,1300,5,7.000,35.0,2023-01-25 08:07:10,2023-10-16 21:29:42,1,1
107,5637171578,8400.0,5637171579,None,None,Medical - Five 5 Days | 2023,2023-01-20,2024-12-31,1300,5,7.000,35.0,2023-01-25 09:10:42,2023-05-25 16:05:53,1,1
178,5637176922,NaN,5637176922,None,None,Life - Five 5 Days | 2024,2024-01-25,2025-12-31,1100,5,7.000,35.0,2024-04-07 00:13:00,2024-04-07 00:13:00,0,0
119,5637172434,NaN,5637172449,None,None,Hala Seha 6 Days | 2023,2023-03-11,2023-05-03,1100,6,6.667,40.0,2023-05-04 16:53:59,2023-08-22 19:50:54,0,0
49,5637170844,10000.0,5637170846,None,None,World Cup Pro Plan 7 Days | 2022,2022-11-17,2023-12-31,1000,7,5.000,35.0,2022-11-15 09:11:25,2022-11-15 09:28:11,1,0
111,5637172399,7700.0,5637172419,None,None,Ramadan Full Plan 7 Days | 2023,2023-03-23,2023-05-01,1000,7,7.714,54.0,2023-03-08 14:35:58,2023-08-22 19:50:54,0,0
115,5637172403,7800.0,5637172423,None,None,Ramadan Iftar & Ghabga Plan 7 Days | 2023,2023-03-23,2023-04-23,750,7,5.571,39.0,2023-03-08 14:44:35,2023-08-22 19:50:54,0,0
185,5637176933,13921.0,5637176937,None,None,Hala Seha 5 | 7 Days - 2024,2024-02-24,2024-12-31,1100,7,6.429,45.0,2024-02-28 13:29:06,2024-12-12 14:11:34,0,0


In [100]:
# remove diet programs with program days = 1
df_diet_programs = df_diet_programs[df_diet_programs['program_days'] > 1]

In [101]:
df_diet_programs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 235 entries, 0 to 234
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id                        235 non-null    int64         
 1   master_plan_id            211 non-null    float64       
 2   diet_program_calender_id  235 non-null    int64         
 3   related_item_id           0 non-null      object        
 4   name                      0 non-null      object        
 5   name_en                   235 non-null    object        
 6   valid_from                235 non-null    object        
 7   valid_to                  235 non-null    object        
 8   calories_total            235 non-null    int64         
 9   program_days              235 non-null    int64         
 10  day_price                 235 non-null    float64       
 11  total_amount              235 non-null    float64       
 12  created_at                2

In [102]:
# drop null's columns (name, related_item_id)
df_diet_programs.drop(columns=['name', 'related_item_id'], inplace=True)

In [103]:
df_diet_programs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 235 entries, 0 to 234
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id                        235 non-null    int64         
 1   master_plan_id            211 non-null    float64       
 2   diet_program_calender_id  235 non-null    int64         
 3   name_en                   235 non-null    object        
 4   valid_from                235 non-null    object        
 5   valid_to                  235 non-null    object        
 6   calories_total            235 non-null    int64         
 7   program_days              235 non-null    int64         
 8   day_price                 235 non-null    float64       
 9   total_amount              235 non-null    float64       
 10  created_at                235 non-null    datetime64[ns]
 11  updated_at                235 non-null    datetime64[ns]
 12  is_renewable              2

In [104]:
# convert start_date and end_date to datetime
df_diet_programs['valid_from'] = pd.to_datetime(df_diet_programs['valid_from'], format='%Y-%m-%d')
df_diet_programs['valid_to'] = pd.to_datetime(df_diet_programs['valid_to'], format='%Y-%m-%d')
df_diet_programs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 235 entries, 0 to 234
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id                        235 non-null    int64         
 1   master_plan_id            211 non-null    float64       
 2   diet_program_calender_id  235 non-null    int64         
 3   name_en                   235 non-null    object        
 4   valid_from                235 non-null    datetime64[ns]
 5   valid_to                  235 non-null    datetime64[ns]
 6   calories_total            235 non-null    int64         
 7   program_days              235 non-null    int64         
 8   day_price                 235 non-null    float64       
 9   total_amount              235 non-null    float64       
 10  created_at                235 non-null    datetime64[ns]
 11  updated_at                235 non-null    datetime64[ns]
 12  is_renewable              2

In [105]:
# validate total_amount depending on program_days and day_price
df_diet_programs['new_total_amount'] = round(df_diet_programs['program_days'] * df_diet_programs['day_price'])
df_diet_programs[df_diet_programs['total_amount'] != df_diet_programs['new_total_amount']]

,id,master_plan_id,diet_program_calender_id,name_en,valid_from,valid_to,calories_total,program_days,day_price,total_amount,created_at,updated_at,is_renewable,display_order,new_total_amount
0,5637163328,8700.0,5637164078,Healthy Keto 30 Days,2022-01-01,2024-12-31,1500,30,9.167,275.001,2022-05-26 11:02:00,2023-05-04 17:18:30,1,0,275.0
4,5637167078,11500.0,5637167078,Life Full Plan 30 Days | 2022,2022-05-02,2023-12-31,1000,30,7.833,234.999,2022-05-11 00:31:31,2023-01-25 18:33:43,1,0,235.0
6,5637167080,13800.0,5637167080,SAYF Plan 26 Days | 2022,2022-05-02,2022-09-18,1000,26,5.962,154.999,2022-05-11 00:31:31,2024-02-23 22:02:01,1,0,155.0
7,5637167081,NaN,5637167081,SAYF Plan 30 Days | 2022,2022-05-02,2022-09-18,1000,30,5.633,168.999,2022-05-11 00:31:31,2023-04-13 13:52:19,1,0,169.0
12,5637167101,10300.0,5637167103,Life Dynamic Four 26 Days | 2022,2022-05-24,2023-12-31,900,26,6.154,159.999,2022-05-21 11:29:04,2023-01-25 18:33:44,1,0,160.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,5637180589,13939.0,5637181341,Life 4L | 26 Days 2025,2024-12-22,2026-12-31,1100,26,4.192,125.000,2024-12-12 13:23:40,2025-01-22 14:31:02,1,0,109.0
231,5637180598,13944.0,5637181350,Life 3 | 26 Days 2025,2024-12-22,2026-12-31,950,26,3.423,102.000,2024-12-19 15:15:29,2025-01-22 14:29:28,1,0,89.0
232,5637180599,13944.0,5637181351,Life 3 | 20 Days 2025,2024-12-22,2026-12-31,950,20,3.950,91.000,2024-12-19 12:29:33,2025-01-22 14:29:19,1,0,79.0
233,5637180600,13943.0,5637181352,Life 4 | 20 Days 2025,2024-12-22,2026-12-31,1000,20,4.700,108.000,2024-12-19 12:23:39,2025-01-22 14:30:56,1,0,94.0


In [106]:
df_diet_programs[ abs(df_diet_programs['total_amount'] - df_diet_programs['new_total_amount']) > 1]

,id,master_plan_id,diet_program_calender_id,name_en,valid_from,valid_to,calories_total,program_days,day_price,total_amount,created_at,updated_at,is_renewable,display_order,new_total_amount
31,5637169334,13400.0,5637169335,Sports 100 plan three meals & snack I 22 days ...,2022-09-14,2023-12-31,1200,22,7.227,180.0,2022-09-15 21:38:55,2022-10-02 09:15:43,1,0,159.0
114,5637172402,NaN,5637172422,Ramadan Full Plan 30 Days | 2023,2023-03-22,2023-05-01,1000,30,6.833,200.0,2023-03-08 14:33:46,2024-02-24 15:41:32,1,0,205.0
118,5637172406,NaN,5637172426,Ramadan Iftar & Ghabga Plan 30 Days | 2023,2023-03-22,2023-05-01,750,30,4.833,155.0,2023-03-08 14:42:26,2024-02-24 15:41:59,1,0,145.0
159,5637176848,13911.0,5637176844,LOW CARB 5 | 22 Days,2023-12-01,2025-02-28,1000,22,6.591,125.0,2023-11-25 08:58:07,2025-01-22 14:35:23,1,0,145.0
160,5637176852,13911.0,5637176848,LOW CARB 5 | 26 Days,2023-12-01,2025-05-31,1000,26,6.346,148.0,2023-11-25 08:54:17,2025-01-22 14:35:14,1,0,165.0
217,5637180576,13941.0,5637181328,Life 6 | 26 Days 2025,2024-12-22,2026-12-31,1900,26,5.731,171.0,2024-12-12 13:23:40,2025-01-22 14:32:47,1,0,149.0
218,5637180577,13941.0,5637181329,Life 6 | 20 Days 2025,2024-12-22,2026-12-31,1900,20,6.200,143.0,2024-12-12 13:23:40,2025-01-22 14:32:37,1,0,124.0
219,5637180578,13942.0,5637181330,Life 7 | 26 Days 2025,2024-12-22,2026-12-31,2000,26,6.500,194.0,2024-12-12 13:23:40,2025-01-22 14:33:18,1,0,169.0
220,5637180579,13942.0,5637181331,Life 7 | 20 Days 2025,2024-12-22,2026-12-31,2000,20,6.950,160.0,2024-12-12 13:23:40,2025-01-22 14:33:11,1,0,139.0
221,5637180580,13940.0,5637181332,Life 5 | 20 Days 2025,2024-12-22,2026-12-31,1000,20,5.450,125.0,2024-12-12 13:23:40,2025-01-22 14:32:05,1,0,109.0


### Validate Price and Return Programs which have issues in pricing of its

In [107]:
msk1 = abs(df_diet_programs['total_amount'] - df_diet_programs['new_total_amount']) > 1
msk2 = df_diet_programs['total_amount'] < df_diet_programs['new_total_amount']
df_diet_programs[ msk1 & msk2 ]

,id,master_plan_id,diet_program_calender_id,name_en,valid_from,valid_to,calories_total,program_days,day_price,total_amount,created_at,updated_at,is_renewable,display_order,new_total_amount
114,5637172402,NaN,5637172422,Ramadan Full Plan 30 Days | 2023,2023-03-22,2023-05-01,1000,30,6.833,200.0,2023-03-08 14:33:46,2024-02-24 15:41:32,1,0,205.0
159,5637176848,13911.0,5637176844,LOW CARB 5 | 22 Days,2023-12-01,2025-02-28,1000,22,6.591,125.0,2023-11-25 08:58:07,2025-01-22 14:35:23,1,0,145.0
160,5637176852,13911.0,5637176848,LOW CARB 5 | 26 Days,2023-12-01,2025-05-31,1000,26,6.346,148.0,2023-11-25 08:54:17,2025-01-22 14:35:14,1,0,165.0


In [108]:
df_diet_programs_with_master_plans = df_diet_programs.merge(
    df_master_plans[['id', 'name_en']],
    left_on='master_plan_id',
    right_on='id',
    how='left'
).rename(columns={'name_en': 'master_plan_name', 'name_en_y': 'master_plan_name'}).drop('id_y', axis=1)
df_diet_programs_with_master_plans

,id_x,master_plan_id,diet_program_calender_id,name_en_x,valid_from,valid_to,calories_total,program_days,day_price,total_amount,created_at,updated_at,is_renewable,display_order,new_total_amount,master_plan_name
0,5637163328,8700.0,5637164078,Healthy Keto 30 Days,2022-01-01,2024-12-31,1500,30,9.167,275.001,2022-05-26 11:02:00,2023-05-04 17:18:30,1,0,275.0,Healthy Keto
1,5637163329,8700.0,5637164079,Healthy Keto 15 Days,2022-01-01,2024-12-31,1500,15,9.933,149.000,2022-05-26 11:02:00,2023-05-04 17:18:30,1,0,149.0,Healthy Keto
2,5637167076,11500.0,5637167076,Life Full Plan 22 Days | 2022,2022-05-02,2023-12-31,1000,22,8.409,185.000,2022-05-11 00:31:31,2023-01-25 18:33:43,1,0,185.0,Life - Full
3,5637167077,11500.0,5637167077,Life Full Plan 26 Days | 2022,2022-05-02,2023-12-31,1000,26,7.885,205.000,2022-05-11 00:31:31,2023-01-25 18:33:43,1,0,205.0,Life - Full
4,5637167078,11500.0,5637167078,Life Full Plan 30 Days | 2022,2022-05-02,2023-12-31,1000,30,7.833,234.999,2022-05-11 00:31:31,2023-01-25 18:33:43,1,0,235.0,Life - Full
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,5637180589,13939.0,5637181341,Life 4L | 26 Days 2025,2024-12-22,2026-12-31,1100,26,4.192,125.000,2024-12-12 13:23:40,2025-01-22 14:31:02,1,0,109.0,Life 4 - Snack
231,5637180598,13944.0,5637181350,Life 3 | 26 Days 2025,2024-12-22,2026-12-31,950,26,3.423,102.000,2024-12-19 15:15:29,2025-01-22 14:29:28,1,0,89.0,Life 3
232,5637180599,13944.0,5637181351,Life 3 | 20 Days 2025,2024-12-22,2026-12-31,950,20,3.950,91.000,2024-12-19 12:29:33,2025-01-22 14:29:19,1,0,79.0,Life 3
233,5637180600,13943.0,5637181352,Life 4 | 20 Days 2025,2024-12-22,2026-12-31,1000,20,4.700,108.000,2024-12-19 12:23:39,2025-01-22 14:30:56,1,0,94.0,Life 4


In [109]:
# save the diet programs to csv
df_diet_programs_with_master_plans.to_csv('../2.clean-data/diet_programs_cleaned.csv', index=False)

### Working on diet programs calories

In [110]:
df_diet_programs_calories = pd.read_sql_query("SELECT * FROM diet_program_calories", engine)
df_diet_programs_calories

,id,diet_program_id,distribution_id,program_code,description,description_en,calories_total,is_visible,created_at,updated_at
0,5637144577,5637144576,000036,CL0014,None,Go Healthy Without Snack 500 Cal,500,0,2022-05-11 00:37:41,2022-05-11 00:37:41
1,5637144578,5637144579,000037,CL0012,None,Go Healthy With Snack 600 Cal,600,0,2022-05-11 00:37:41,2022-05-11 00:37:41
2,5637144579,5637144610,000038,CL0015,None,Go Healthy Without Snack 500 Cal,500,0,2022-05-11 00:37:41,2022-05-11 00:37:41
3,5637144580,5637144609,000039,CL0013,None,Go Healthy With Snack 600 Cal,600,0,2022-05-11 00:37:41,2022-05-11 00:37:41
4,5637144581,5637144580,000040,CL0019,None,ProHealthy Without Snack 850 Cal,850,0,2022-05-11 00:37:42,2022-05-11 00:37:42
...,...,...,...,...,...,...,...,...,...,...
1747,5637186592,5637180600,002351,CL00526,None,Life 4 | 20 Days 2025,1100,1,2024-12-19 12:26:36,2024-12-19 12:26:36
1748,5637186593,5637180600,002352,CL00526,None,Life 4 | 20 Days 2025,1250,1,2024-12-19 12:26:36,2024-12-19 12:26:36
1749,5637186594,5637180601,002353,CL00527,None,Life 4 | 26 Days 2025,1000,1,2024-12-19 12:29:31,2024-12-19 12:29:31
1750,5637186595,5637180601,002354,CL00527,None,Life 4 | 26 Days 2025,1100,1,2024-12-19 12:29:31,2024-12-19 12:29:31


In [111]:
df_diet_programs_calories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1752 entries, 0 to 1751
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               1752 non-null   int64         
 1   diet_program_id  1752 non-null   int64         
 2   distribution_id  1752 non-null   object        
 3   program_code     1752 non-null   object        
 4   description      0 non-null      object        
 5   description_en   1752 non-null   object        
 6   calories_total   1752 non-null   int64         
 7   is_visible       1752 non-null   int64         
 8   created_at       1752 non-null   datetime64[ns]
 9   updated_at       1752 non-null   datetime64[ns]
dtypes: datetime64[ns](2), int64(4), object(4)
memory usage: 137.0+ KB


In [112]:
# select only important columns
df_diet_programs_calories = pd.read_sql_query("SELECT id, diet_program_id, description_en, calories_total, is_visible FROM diet_program_calories", engine)
df_diet_programs_calories

,id,diet_program_id,description_en,calories_total,is_visible
0,5637144577,5637144576,Go Healthy Without Snack 500 Cal,500,0
1,5637144578,5637144579,Go Healthy With Snack 600 Cal,600,0
2,5637144579,5637144610,Go Healthy Without Snack 500 Cal,500,0
3,5637144580,5637144609,Go Healthy With Snack 600 Cal,600,0
4,5637144581,5637144580,ProHealthy Without Snack 850 Cal,850,0
...,...,...,...,...,...
1747,5637186592,5637180600,Life 4 | 20 Days 2025,1100,1
1748,5637186593,5637180600,Life 4 | 20 Days 2025,1250,1
1749,5637186594,5637180601,Life 4 | 26 Days 2025,1000,1
1750,5637186595,5637180601,Life 4 | 26 Days 2025,1100,1


In [113]:
df_diet_programs_calories.duplicated().sum()

0

In [114]:
# check if the diet_program with calories total duplicated or not
df_diet_programs_calories[ ['diet_program_id', 'calories_total']].duplicated().sum()

11

In [115]:
# get the duplicated diet_program_calories depending on diet_program_id and calories_total
df_diet_programs_calories[df_diet_programs_calories.duplicated(subset=['diet_program_id', 'calories_total'], keep=False)].sort_values(['diet_program_id', 'calories_total'])

,id,diet_program_id,description_en,calories_total,is_visible
10,5637144588,5637144591,Best Life 1000 Cal,1000,0
157,5637146826,5637144591,Best Life 1100 Cal,1000,0
170,5637147577,5637144593,Bariatric Prepureed 1000 cal,1000,0
1419,5637177740,5637144593,Bariatric Pureed 7 Days,1000,0
171,5637147578,5637144594,Bariatric Pureed 1000 Cal,1000,0
1250,5637176076,5637144594,Bariatric Mashed 14 Days,1000,0
146,5637146076,5637144595,Bariatric soft phase 1000 cal,1000,0
1251,5637176077,5637144595,Bariatric Soft phase 1000 cal,1000,0
1218,5637174579,5637169327,Wanasa Plan 26 Days | 2022,1100,1
1263,5637176093,5637169327,Wanasa Plan 26 Days | 2022,1100,0


In [116]:
df_diet_programs_calories.describe()

,id,diet_program_id,calories_total,is_visible
count,1.752000e+03,1.752000e+03,1752.000000,1752.000000
mean,5.637166e+09,5.637161e+09,1386.679224,0.264840
std,1.154334e+04,1.082735e+04,620.107594,0.441374
min,5.637145e+09,5.637145e+09,1.000000,0.000000
25%,5.637156e+09,5.637151e+09,1100.000000,0.000000
50%,5.637166e+09,5.637160e+09,1300.000000,0.000000
75%,5.637176e+09,5.637171e+09,1600.000000,1.000000
max,5.637187e+09,5.637181e+09,20000.000000,1.000000


In [117]:
# visualize calories total depending on is_visible
px.pie(
    df_diet_programs_calories,
    names='is_visible',
    title='Percentage of Diet Program Calories Visibility'
)

In [118]:
# visualize total calories by diet programs
px.histogram(
    df_diet_programs_calories,
    x='calories_total',
    title='Total Calories by Diet Programs',
    text_auto=True
).update_layout(bargap=0.2)

In [119]:
# detect outliers in calories_total
df_diet_programs_calories_outliers = detect_outliers_iqr(df_diet_programs_calories, 'calories_total').sort_values('calories_total')
df_diet_programs_calories_outliers['calories_total'].value_counts()

calories_total
1        12
2400      9
2500      9
2600      9
2700      9
2800      9
2900      9
3000      9
20000     1
Name: count, dtype: int64

In [120]:
# working on diet programs which have calories total = 20000
diet_program_id = df_diet_programs_calories[df_diet_programs_calories['calories_total'] == 20000].reset_index()['diet_program_id'][0]
df_diet_programs_calories[df_diet_programs_calories['diet_program_id'] == diet_program_id]

,id,diet_program_id,description_en,calories_total,is_visible
708,5637164828,5637159583,Best life 35 day Jahra Offer 2020,1000,0
709,5637164829,5637159583,Best life 35 day Jahra Offer 2020,1100,0
710,5637164830,5637159583,Best life 35 day Jahra Offer 2020,1200,0
711,5637164831,5637159583,Best life 35 day Jahra Offer 2020,1300,0
712,5637164832,5637159583,Best life 35 day Jahra Offer 2020,1400,0
713,5637164833,5637159583,Best life 35 day Jahra Offer 2020,1500,0
714,5637164834,5637159583,Best life 35 day Jahra Offer 2020,1600,0
715,5637164835,5637159583,Best life 35 day Jahra Offer 2020,1700,0
716,5637164836,5637159583,Best life 35 day Jahra Offer 2020,1800,0
717,5637164837,5637159583,Best life 35 day Jahra Offer 2020,1900,0


In [121]:
# i'll replace the calories total for this diet program with 2000
df_diet_programs_calories.loc[df_diet_programs_calories['diet_program_id'] == diet_program_id, 'calories_total'] = 2000
# visualize total calories by diet programs
px.histogram(
    df_diet_programs_calories,
    x='calories_total',
    title='Total Calories by Diet Programs',
    text_auto=True
).update_layout(bargap=0.2)

In [122]:
# working on diet programs which have calories total = 1
diet_programs_ids = df_diet_programs_calories[df_diet_programs_calories['calories_total'] == 1].reset_index()['diet_program_id'].tolist()
df_diet_programs_calories[df_diet_programs_calories['diet_program_id'].isin(diet_programs_ids)]


,id,diet_program_id,description_en,calories_total,is_visible
1504,5637179851,5637174592,Easy Plan (2 Salads and 2 Main Courses) 10 Day...,1,1
1505,5637179852,5637174593,Easy Plan (2 Salads and 2 Main Courses) 12 Day...,1,1
1506,5637179853,5637174594,Easy Plan (2 Salads and 2 Main Courses) 20 Day...,1,1
1507,5637179854,5637174595,Easy Plan (2 Salads and 2 Main Courses) 22 Day...,1,1
1508,5637179855,5637174596,Easy Plan (Salad and 2 Main Courses) 10 Days |...,1,1
1509,5637179857,5637174597,Easy Plan (Salad and 2 Main Courses) 12 Days |...,1,1
1510,5637179858,5637174598,Easy Plan (Salad and 2 Main Courses) 20 Days |...,1,1
1511,5637179859,5637174599,Easy Plan (Salad and 2 Main Courses) 22 Days |...,1,1
1512,5637179860,5637174600,Easy Plan (Salad and Main Course) 10 Days | 2023,1,1
1513,5637179861,5637174601,Easy Plan (Salad and Main Course) 12 Days | 2023,1,1


In [123]:
df_diet_programs_calories[df_diet_programs_calories['diet_program_id'] == 5637174593]

,id,diet_program_id,description_en,calories_total,is_visible
1505,5637179852,5637174593,Easy Plan (2 Salads and 2 Main Courses) 12 Day...,1,1


In [124]:
df_diet_programs[df_diet_programs['id'].isin(diet_programs_ids)].count()
# this programs didn't exist so i'll remove them from diet programs calories table

id                          0
master_plan_id              0
diet_program_calender_id    0
name_en                     0
valid_from                  0
valid_to                    0
calories_total              0
program_days                0
day_price                   0
total_amount                0
created_at                  0
updated_at                  0
is_renewable                0
display_order               0
new_total_amount            0
dtype: int64

In [125]:
# remove all diet programs calories which have calories total = 1
df_diet_programs_calories = df_diet_programs_calories[~df_diet_programs_calories['diet_program_id'].isin(diet_programs_ids)]

In [126]:
px.histogram(
    df_diet_programs_calories,
    x='calories_total',
    title='Total Calories by Diet Programs',
    text_auto=True
).update_layout(bargap=0.2)

### Diet Programs Meals

In [127]:
df_diet_programs_meals = pd.read_sql_query("SELECT * FROM diet_program_meals", engine)
df_diet_programs_meals

,id,diet_program_id,program_code,meal_code,name,name_en,portion,created_at,updated_at
0,5637144980,5637144576,CL0014,CAT02,السلطة والشوربة للغداء,Salad And Soup For Lunch,1.0,2022-05-11 00:32:17,2022-05-11 00:32:17
1,5637144981,5637144579,CL0012,CAT01,منتجات الحليب,Beverage,1.0,2022-05-11 00:32:17,2022-05-11 00:32:17
2,5637144982,5637144579,CL0012,CAT02,الغداء,Lunch,1.0,2022-05-11 00:32:17,2022-05-11 00:32:17
3,5637144983,5637144579,CL0012,CAT03,السلطة والشوربة للغداء,Salad And Soup For Lunch,1.0,2022-05-11 00:32:17,2022-05-11 00:32:17
4,5637144984,5637144579,CL0012,CAT04,سناك المساء,Evening Snack,1.0,2022-05-11 00:32:17,2022-05-11 00:32:17
...,...,...,...,...,...,...,...,...,...
2701,5637181330,5637181326,CL00528,CAT01,الأفطار,Breakfast,1.0,2024-12-17 23:00:23,2024-12-17 23:00:23
2702,5637181331,5637181326,CL00528,CAT02,الغداء,Lunch,1.0,2024-12-17 23:00:23,2024-12-17 23:00:23
2703,5637181332,5637181326,CL00528,CAT03,العشاء,Dinner,1.0,2024-12-17 23:00:23,2024-12-17 23:00:23
2704,5637181333,5637181326,CL00528,CAT04,طبق جانبي 1,Side Dish 1,1.0,2024-12-17 23:00:23,2024-12-17 23:00:23


In [128]:
# select only important columns
df_diet_programs_meals = pd.read_sql_query("SELECT id, diet_program_id, meal_code, name, name_en, portion FROM diet_program_meals", engine)
df_diet_programs_meals

,id,diet_program_id,meal_code,name,name_en,portion
0,5637144980,5637144576,CAT02,السلطة والشوربة للغداء,Salad And Soup For Lunch,1.0
1,5637144981,5637144579,CAT01,منتجات الحليب,Beverage,1.0
2,5637144982,5637144579,CAT02,الغداء,Lunch,1.0
3,5637144983,5637144579,CAT03,السلطة والشوربة للغداء,Salad And Soup For Lunch,1.0
4,5637144984,5637144579,CAT04,سناك المساء,Evening Snack,1.0
...,...,...,...,...,...,...
2701,5637181330,5637181326,CAT01,الأفطار,Breakfast,1.0
2702,5637181331,5637181326,CAT02,الغداء,Lunch,1.0
2703,5637181332,5637181326,CAT03,العشاء,Dinner,1.0
2704,5637181333,5637181326,CAT04,طبق جانبي 1,Side Dish 1,1.0


In [129]:
df_diet_programs_meals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2706 entries, 0 to 2705
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               2706 non-null   int64  
 1   diet_program_id  2706 non-null   int64  
 2   meal_code        2706 non-null   object 
 3   name             2706 non-null   object 
 4   name_en          2706 non-null   object 
 5   portion          2706 non-null   float64
dtypes: float64(1), int64(2), object(3)
memory usage: 127.0+ KB


In [130]:
df_diet_programs_meals.duplicated().sum()

0

In [131]:
df_diet_programs_meals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2706 entries, 0 to 2705
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               2706 non-null   int64  
 1   diet_program_id  2706 non-null   int64  
 2   meal_code        2706 non-null   object 
 3   name             2706 non-null   object 
 4   name_en          2706 non-null   object 
 5   portion          2706 non-null   float64
dtypes: float64(1), int64(2), object(3)
memory usage: 127.0+ KB


In [132]:
# merge diet programs with diet programs meals
df_diet_programs_meals = pd.merge(left=df_diet_programs_meals,
    right=df_diet_programs[['id', 'name_en', 'program_days']],
    left_on='diet_program_id',
    right_on='id',
    how='inner',
    suffixes=('', '_program')
).drop(['id_program', 'name'], axis=1)
df_diet_programs_meals

,id,diet_program_id,meal_code,name_en,portion,name_en_program,program_days
0,5637163339,5637163328,CAT01,Breakfast,1.0,Healthy Keto 30 Days,30
1,5637163340,5637163328,CAT02,Lunch,1.0,Healthy Keto 30 Days,30
2,5637163345,5637163328,CAT03,Dinner,1.0,Healthy Keto 30 Days,30
3,5637163346,5637163328,CAT04,Salad Or Soup,1.0,Healthy Keto 30 Days,30
4,5637163347,5637163328,CAT05,Snack,1.0,Healthy Keto 30 Days,30
...,...,...,...,...,...,...,...
1091,5637180686,5637180600,CAT04,Side Dish,1.0,Life 4 | 20 Days 2025,20
1092,5637181326,5637180601,CAT01,Breakfast Or Side Dish,1.0,Life 4 | 26 Days 2025,26
1093,5637181327,5637180601,CAT02,Lunch,1.0,Life 4 | 26 Days 2025,26
1094,5637181328,5637180601,CAT03,Dinner,1.0,Life 4 | 26 Days 2025,26


In [133]:
# number of meals in each diet program
programs_with_meals_days_count = df_diet_programs_meals.groupby(['diet_program_id', 'name_en_program', 'program_days']).size().reset_index(name='meals_count')
programs_with_meals_days_count

,diet_program_id,name_en_program,program_days,meals_count
0,5637163328,Healthy Keto 30 Days,30,5
1,5637163329,Healthy Keto 15 Days,15,5
2,5637167076,Life Full Plan 22 Days | 2022,22,7
3,5637167077,Life Full Plan 26 Days | 2022,26,7
4,5637167078,Life Full Plan 30 Days | 2022,30,7
...,...,...,...,...
228,5637180589,Life 4L | 26 Days 2025,26,4
229,5637180598,Life 3 | 26 Days 2025,26,3
230,5637180599,Life 3 | 20 Days 2025,20,3
231,5637180600,Life 4 | 20 Days 2025,20,4


In [134]:
# check if there is a correlation between program days and meals count
programs_with_meals_days_count.describe()

,diet_program_id,program_days,meals_count
count,2.330000e+02,233.000000,233.000000
mean,5.637173e+09,24.111588,4.703863
std,4.079680e+03,4.967610,1.565477
min,5.637163e+09,5.000000,2.000000
25%,5.637171e+09,22.000000,3.000000
50%,5.637172e+09,26.000000,5.000000
75%,5.637177e+09,26.000000,5.000000
max,5.637181e+09,30.000000,9.000000


In [135]:
px.scatter_matrix(
    programs_with_meals_days_count,
    dimensions=['program_days', 'meals_count'],
    title='Scatter Matrix of Program Days and Meals Count'
)

In [136]:
programs_with_meals_days_count[['program_days', 'meals_count']].corr()

,program_days,meals_count
program_days,1.000000,-0.015686
meals_count,-0.015686,1.000000


In [137]:
# visualize the programs count by meals count
px.histogram(
    programs_with_meals_days_count,
    x='meals_count',
    title='Number of Meals by Diet Programs',
    text_auto=True
).update_layout(bargap=0.2)

In [138]:
# meals count by meal code
px.histogram(
    df_diet_programs_meals,
    x='meal_code',
    title='Number of Meals by Meal Code',
    text_auto=True
).update_layout(bargap=0.2)

In [139]:
# meals count by meal name
px.histogram(
    df_diet_programs_meals,
    x='name_en',
    title='Number of Meals by Meal Name',
    text_auto=True
).update_layout(bargap=0.2)

In [140]:
# convert name_en to small letters
df_diet_programs_meals['name_en'] = df_diet_programs_meals['name_en'].str.lower()

In [141]:
df_diet_programs_meals['name_en'].value_counts()

name_en
lunch                       151
dinner                      146
breakfast                   116
snack                        72
salad or soup                65
meal 1                       60
meal 2                       60
morning snack                38
salad or soup for lunch      38
salad or soup for dinner     38
meal 3                       36
evening snack                36
side dish                    30
ghabqa                       19
futour                       19
beverage                     19
suhour                       12
breakfast or snack           12
breakfast or side dish       11
fruit                        11
appetizer                    11
salad or soup for futour     11
snack 2                      10
snack 1                      10
salad / soup                  7
salad                         7
meal 4                        4
soup                          4
snack 3                       4
breakfast / snack             3
salad or soup for suhour      3


In [142]:
# get all snacks meals
snack_meals = df_diet_programs_meals[df_diet_programs_meals['name_en'].str.contains('snack', case=False)]
snack_meals

,id,diet_program_id,meal_code,name_en,portion,name_en_program,program_days
4,5637163347,5637163328,CAT05,snack,1.0,Healthy Keto 30 Days,30
9,5637163352,5637163329,CAT05,snack,1.0,Healthy Keto 15 Days,15
11,5637167077,5637167076,CAT02,morning snack,1.0,Life Full Plan 22 Days | 2022,22
14,5637167080,5637167076,CAT05,evening snack,1.0,Life Full Plan 22 Days | 2022,22
18,5637167084,5637167077,CAT02,morning snack,1.0,Life Full Plan 26 Days | 2022,26
...,...,...,...,...,...,...,...
1073,5637180631,5637180588,CAT04,snack 2,1.0,Life 4L | 20 Days 2025,20
1076,5637180634,5637180587,CAT03,snack 1,1.0,Life 4B | 20 Days 2025,20
1077,5637180635,5637180587,CAT04,snack 2,1.0,Life 4B | 20 Days 2025,20
1080,5637180638,5637180589,CAT03,snack 1,1.0,Life 4L | 26 Days 2025,26


In [143]:
# check if programs with snack meals have more than one meal
snack_meals.groupby('diet_program_id').size().reset_index(name='count').query('count > 1')

,diet_program_id,count
2,5637167076,2
3,5637167077,2
4,5637167078,2
8,5637167082,2
9,5637167083,2
10,5637167084,2
13,5637167826,3
14,5637167827,2
15,5637167828,2
40,5637170861,2


In [144]:
snack_meals[snack_meals['diet_program_id'] == 5637180576]

,id,diet_program_id,meal_code,name_en,portion,name_en_program,program_days
1021,5637180579,5637180576,CAT04,snack 1,1.0,Life 6 | 26 Days 2025,26
1022,5637180580,5637180576,CAT05,snack 2,1.0,Life 6 | 26 Days 2025,26
1023,5637180581,5637180576,CAT06,snack 3,1.0,Life 6 | 26 Days 2025,26


In [145]:
df_diet_programs_meals[['diet_program_id', 'meal_code']].duplicated().sum()
# get the duplicated diet_programs_meals depending on diet_program_id and meal_code
df_diet_programs_meals[df_diet_programs_meals.duplicated(subset=['diet_program_id', 'meal_code'], keep=False)].sort_values(['diet_program_id', 'meal_code'])

,id,diet_program_id,meal_code,name_en,portion,name_en_program,program_days
68,5637167830,5637167826,CAT05,evening snack,1.0,LIFE PLAN – FULL MEDICAL (22 DAYS) | 2022,22
116,5637168593,5637167826,CAT05,evening snack,1.0,LIFE PLAN – FULL MEDICAL (22 DAYS) | 2022,22
69,5637167831,5637167826,CAT06,dinner,1.0,LIFE PLAN – FULL MEDICAL (22 DAYS) | 2022,22
115,5637168592,5637167826,CAT06,dinner,1.0,LIFE PLAN – FULL MEDICAL (22 DAYS) | 2022,22


In [146]:
# remove the duplicated diet_programs_meals depending on diet_program_id and meal_code
df_diet_programs_meals = df_diet_programs_meals.drop_duplicates(subset=['diet_program_id', 'meal_code'])

In [147]:
df_diet_programs_meals[['diet_program_id', 'meal_code']].duplicated().sum()

0

In [148]:
df_diet_programs_meals.drop(['name_en_program', 'program_days'], axis=1, inplace=True)
df_diet_programs_meals.duplicated().sum()

0

In [149]:
df_diet_programs_meals

,id,diet_program_id,meal_code,name_en,portion
0,5637163339,5637163328,CAT01,breakfast,1.0
1,5637163340,5637163328,CAT02,lunch,1.0
2,5637163345,5637163328,CAT03,dinner,1.0
3,5637163346,5637163328,CAT04,salad or soup,1.0
4,5637163347,5637163328,CAT05,snack,1.0
...,...,...,...,...,...
1091,5637180686,5637180600,CAT04,side dish,1.0
1092,5637181326,5637180601,CAT01,breakfast or side dish,1.0
1093,5637181327,5637180601,CAT02,lunch,1.0
1094,5637181328,5637180601,CAT03,dinner,1.0


### Working on diet program items & meal items

In [150]:
# select diet_program_items
df_diet_programs_items = pd.read_sql_query("SELECT * FROM diet_program_items", engine)
df_diet_programs_items

,id,name,name_en,image,created_at,updated_at
0,5637179076,بيتزا 4 اجبان,4 Cheese Pizza,None,2022-05-11 00:32:40,2022-05-11 00:32:40
1,5637179077,البثيث,Al Bethith,None,2022-05-11 00:32:40,2022-05-11 00:32:40
2,5637179078,ألبا,Alba,None,2022-05-11 00:32:40,2022-05-11 00:32:40
3,5637179079,لوز وجوز,Almond & Walnut,None,2022-05-11 00:32:40,2022-05-11 00:32:40
4,5637179080,دجاج مع اللوز يقدم مع الأرز الأسمر,Almond Chicken Served With Brown Rice,None,2022-05-11 00:32:40,2022-05-11 00:32:40
...,...,...,...,...,...,...
4112,5637269913,سمكة حارة مع أرز بالزعفران,HP-Samkeh Harra w/ Saffron Rice,None,2024-12-19 15:15:36,2024-12-19 15:15:36
4113,5637269914,بيض مخفوق و بطاطا,Scrambeled egg with potato,None,2024-12-19 15:15:36,2024-12-19 15:15:36
4114,5637269915,بيض مخفوق و بطاطا,Scrambeled Egg w/ Potato,None,2024-12-19 15:15:36,2024-12-19 15:15:36
4115,5637269916,أومليت فيلي-ستيك مع خبز,Philly Steak Omellete w/ Bread,None,2024-12-19 15:15:36,2024-12-19 15:15:36


In [151]:
df_diet_programs_items.drop(columns=['name', 'image', 'created_at', 'updated_at'], inplace=True)

In [152]:
df_diet_programs_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4117 entries, 0 to 4116
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       4117 non-null   int64 
 1   name_en  4117 non-null   object
dtypes: int64(1), object(1)
memory usage: 64.5+ KB


In [153]:
df_diet_programs_items['name_en'].duplicated().sum()

319

In [154]:
# select duplicated diet_program_items depending on name_en
df_diet_programs_items[df_diet_programs_items['name_en'].duplicated(keep=False)].sort_values('name_en')

,id,name_en
2854,5637248983,Alfalfa Pomegranate Salad
1607,5637237602,Alfalfa Pomegranate Salad
17,5637179094,Apple Juice
3408,5637259431,Apple Juice
3170,5637255254,Asian Orange Chicken
...,...,...
3526,5637261036,White Chocolate Cereal
2782,5637248911,White Chocolate Cornflakes
3475,5637260984,White Chocolate Cornflakes
2728,5637248856,Za'atar Fatayer


In [155]:
# diet program items with diet program meals items
df_diet_programs_meals_items = pd.read_sql_query("SELECT * FROM diet_program_meal_items", engine)
df_diet_programs_meals_items

,id,diet_program_id,diet_program_meal_id,diet_program_item_id,program_code,program_day,category_code,created_at,updated_at
0,5637190909,5637144576,5637145326,5637179730,CL0014,1,CAT01,2022-05-12 13:16:56,2022-05-12 13:16:56
1,5637190910,5637144576,5637145326,5637179448,CL0014,1,CAT01,2022-05-12 13:16:56,2022-05-12 13:16:56
2,5637190911,5637144576,5637145326,5637179163,CL0014,1,CAT01,2022-05-12 13:16:56,2022-05-12 13:16:56
3,5637190912,5637144576,5637144980,5637179175,CL0014,1,CAT02,2022-05-12 13:16:56,2022-05-12 13:16:56
4,5637190913,5637144576,5637144980,5637179627,CL0014,1,CAT02,2022-05-12 13:16:56,2022-05-12 13:16:56
...,...,...,...,...,...,...,...,...,...
460135,5638164816,5637181326,5637181334,5637260936,CL00528,3,CAT05,2024-12-17 23:01:29,2024-12-17 23:01:29
460136,5638164817,5637181326,5637181334,5637260936,CL00528,4,CAT05,2024-12-17 23:01:29,2024-12-17 23:01:29
460137,5638164818,5637181326,5637181334,5637260936,CL00528,5,CAT05,2024-12-17 23:01:29,2024-12-17 23:01:29
460138,5638164819,5637181326,5637181334,5637260936,CL00528,6,CAT05,2024-12-17 23:01:29,2024-12-17 23:01:29


In [156]:
df_diet_programs_meals_items.drop(columns=['created_at', 'updated_at'], inplace=True)

In [157]:
df_diet_programs_meals_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460140 entries, 0 to 460139
Data columns (total 7 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   id                    460140 non-null  int64 
 1   diet_program_id       460140 non-null  int64 
 2   diet_program_meal_id  460140 non-null  int64 
 3   diet_program_item_id  460140 non-null  int64 
 4   program_code          460140 non-null  object
 5   program_day           460140 non-null  int64 
 6   category_code         460140 non-null  object
dtypes: int64(5), object(2)
memory usage: 24.6+ MB


In [158]:
# get all diet programs without diet program meals
df_diet_programs[~df_diet_programs['id'].isin(df_diet_programs_meals['diet_program_id'])]


,id,master_plan_id,diet_program_calender_id,name_en,valid_from,valid_to,calories_total,program_days,day_price,total_amount,created_at,updated_at,is_renewable,display_order,new_total_amount
213,5637179092,NaN,5637179092,Protein 150 Two Meals 2024 | 26 Days,2024-04-10,2024-12-31,1200,26,5.385,140.000,2024-12-11 15:52:36,2024-12-12 14:11:34,0,0,140.0
214,5637179093,NaN,5637179093,Protein 150 Two Meals 2024 | 22 Days,2024-04-10,2024-12-31,1200,22,5.455,119.999,2024-12-11 15:52:36,2024-12-12 14:11:34,0,0,120.0


In [159]:
# get all diet programs without diet program meals items
df_diet_programs[~df_diet_programs['id'].isin(df_diet_programs_meals_items['diet_program_id'])]


,id,master_plan_id,diet_program_calender_id,name_en,valid_from,valid_to,calories_total,program_days,day_price,total_amount,created_at,updated_at,is_renewable,display_order,new_total_amount
172,5637176915,NaN,5637176912,Flexible Weight Loss 5 | 22 Days,2023-12-15,2024-12-31,1100,22,5.682,125.000,2024-04-07 00:13:00,2024-12-12 14:11:34,0,0,125.0
173,5637176916,NaN,5637176913,Flexible Weight Loss 5 | 26 Days,2023-12-15,2024-12-31,1100,26,5.385,140.000,2024-04-07 00:13:00,2024-12-12 14:11:34,0,0,140.0
178,5637176922,NaN,5637176922,Life - Five 5 Days | 2024,2024-01-25,2025-12-31,1100,5,7.000,35.000,2024-04-07 00:13:00,2024-04-07 00:13:00,0,0,35.0
213,5637179092,NaN,5637179092,Protein 150 Two Meals 2024 | 26 Days,2024-04-10,2024-12-31,1200,26,5.385,140.000,2024-12-11 15:52:36,2024-12-12 14:11:34,0,0,140.0
214,5637179093,NaN,5637179093,Protein 150 Two Meals 2024 | 22 Days,2024-04-10,2024-12-31,1200,22,5.455,119.999,2024-12-11 15:52:36,2024-12-12 14:11:34,0,0,120.0
